In [1]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from matplotlib import rcParams
import math
import seaborn as sns
from scipy.stats import spearmanr
from typing import List, Optional

Historical stock returns have always been an important reference for future returns. This research utilizes historical time-series price and volume data for all stocks to identify effective candlestick patterns and their compatible market states.

#### Candlestick Pattern Investment Framework

Candlesticks represent the price trajectory formed by the interaction between buyers and sellers during trading, essentially reflecting the path of capital flows. Investment decisions based solely on candlestick patterns are relatively limited. By integrating candlestick patterns with price-volume conditions, the accuracy of predictions and the success rate of investments can be significantly improved. This paper establishes an investment framework based on both candlestick pattern recognition and price-volume state identification.

#### Candlestick Pattern and Price-Volume State Identification

This study will primarily use the following candlestick features for pattern recognition: Bullish/Bearish (Yin & Yang) attribute, Body ratio, Upper Shadow ratio, Lower Shadow ratio, Returns, Open, High, Low, and Close prices. Regarding price-volume states, the most frequently occurring and widely used conditions are high volume and low volume, combined with the stock price being at a relative high or low level. To ensure broad applicability, this paper will use these two types of price-volume conditions to construct the identification framework.

In [2]:
all_stock_daily = pd.read_parquet('data/stock_daily.parquet')

In [3]:
all_stock_daily

open   high    low  close  pre_close  change  pct_chg  \
trade_date ts_code                                                             
2014-01-02 000001.SZ   5.37   5.45   5.34   5.42       5.43   -0.01    -0.18   
           000002.SZ   5.14   5.19   5.09   5.14       5.16   -0.02    -0.39   
           000004.SZ  11.64  11.87  11.45  11.81      11.65    0.16     1.37   
           000006.SZ   3.82   3.83   3.75   3.77       3.82   -0.05    -1.31   
           000007.SZ   9.26   9.52   9.15   9.36       9.25    0.11     1.19   
...                     ...    ...    ...    ...        ...     ...      ...   
2025-09-26 920445.BJ  11.02  11.43  11.02  11.06      11.02    0.04     0.36   
           920489.BJ  23.46  23.76  22.54  22.57      23.72   -1.15    -4.85   
           920682.BJ   9.82  10.25   9.81  10.08       9.88    0.20     2.02   
           920799.BJ  55.86  55.92  54.50  54.50      55.22   -0.72    -1.30   
           920819.BJ   4.25   4.31   4.20   4.25       4.24    0.01     0.24   

                            vol       amount  
trade_date ts_code                            
2014-01-02 000001.SZ  489910.89  596223.7444  
           000002.SZ  485299.22  387391.0664  
           000004.SZ    9714.74   11417.5556  
           000006.SZ   88230.28   43051.9277  
           000007.SZ   32811.75   46266.7890  
...                         ...          ...  
2025-09-26 920445.BJ   19097.22   21424.6980  
           920489.BJ   73613.53  168578.2070  
           920682.BJ   60706.91   61218.3150  
           920799.BJ   23112.55  127756.1470  
           920819.BJ   85665.17   36397.3400  

[10612042 rows x 9 columns]

In [4]:
# checking for missing values in the data
all_stock_daily.isna().sum()

open           0
high           0
low            0
close          0
pre_close    233
change       233
pct_chg      234
vol            0
amount         0
dtype: int64

In [5]:
all_stock_daily.dropna(inplace=True)

In [6]:
all_stock_daily.isna().sum()

open         0
high         0
low          0
close        0
pre_close    0
change       0
pct_chg      0
vol          0
amount       0
dtype: int64

In [7]:
all_stocks = all_stock_daily.index.get_level_values('ts_code').unique().to_list()
all_trade_dates = all_stock_daily.index.get_level_values('trade_date').unique().to_list()

# 1 $\,$ Candlestick Pattern Investment Framework

## 1.1 $\,$ Candlestick Pattern Recognition

Candlesticks are drawn based on the opening, highest, lowest, and closing prices of the analysis period. Taking the daily candlestick as an example, the section between the opening price and the closing price forms the candlestick's real body. If the closing price is higher than the opening price, it is called a bullish candlestick (Yang line), represented by a hollow (or red) body; conversely, it is called a bearish candlestick (Yin line), represented by a filled (or green) body. The line between the highest price and the real body is called the upper shadow, and the line between the lowest price and the real body is called the lower shadow.

<div align="center">
<img src="./img/1.png" width="800">
</div>

To meet the requirements of candlestick pattern recognition, the basic features of the candlestick need to be quantified. The Amplitude of stock i at time t refers to the difference between the high and low prices divided by the previous closing price.

$$ \text { Amplitude }_{i, t}=\left(\text { High }_{i, t}-\text { Low }_{i, t}\right) / \text { PrevClose }_{i, t} $$

Effective candlestick patterns often originate from traders' long-term observation of candlesticks during trading. In practical application, the relative proportions of the real body, upper shadow, and lower shadow to the amplitude are closer to the actual observation perspective of traders and are more intuitive and convenient for candlestick pattern recognition. Therefore, we will use the relative proportions of the real body, upper shadow, and lower shadow to the amplitude to characterize the candlestick features of a stock, thereby completing candlestick pattern recognition.

The real body refers to the absolute difference between the opening and closing prices divided by the previous closing price. The Body ratio of stock i at time t refers to the relative proportion of the real body to the amplitude.

$$ \text { Body }{ }_{i, t}=\operatorname{abs}\left(\text { Close }_{i, t}-\text { Open }_{i, t}\right) / \text { PrevClose }_{i, t} / \text { Amplitude }_{i, t} $$

The upper shadow refers to the difference between the highest price and the maximum of the opening and closing prices, divided by the previous closing price. The Upper Shadow ratio of stock i at time t refers to the ratio of the upper shadow to the amplitude.

$$ \text { Upper Shadow }{ }_{i, t}=\left(\text { High }_{i, t}-\max \left(\text { Open }_{i, t}, \text { Close }_{i, t}\right)\right) / \text { PrevClose }_{i, t} / \text { Amplitude }_{i, t} $$

The lower shadow refers to the difference between the minimum of the opening and closing prices and the lowest price, divided by the previous closing price. The Lower Shadow ratio of stock i at time t refers to the ratio of the lower shadow to the amplitude.

$$ \text { Upper Shadow }_{i,t} = \left(\min\left(\text { Open }_{i,t}, \text { Close }_{i,t}\right)-\text { Low }_{i,t}\right)/\text { PrevClose }_{i,t}/\text { Amplitude }_{i,t} $$

This paper will primarily use candlestick features such as the Bullish/Bearish attribute (Yin & Yang), Body ratio, Upper Shadow ratio, Lower Shadow ratio, Returns, Open price, High price, Low price, and Close price for pattern recognition.

The results of candlestick pattern recognition are organized into 0-1 variables. The candlestick feature symbols used hereafter are consistent with those in this section and will not be reiterated.

## 1.2 $\,$ Price-Volume State Identification

Among stock price-volume states, the most frequently occurring and widely used conditions are high volume versus low volume, and the stock price being at a relative high or low level. To ensure broad applicability, this paper will use these two types of price-volume conditions to construct the stock's price-volume state identification information.

The results of both volume state identification and stock price state identification are organized into 0-1 variables. The results of candlestick pattern recognition and price-volume state identification are aggregated via set intersection.

## 1.2.1 $\,$ Volume State Identification

High volume and low volume are defined by the relative change in volume between two consecutive days. If the volume of stock i at time t is greater than its volume at time t-1, it is in a high volume state (Volume Expansion, 1). Conversely, if the volume of stock i at time t is less than or equal to the volume on day t-1, it is in a low volume state (Volume Contraction, 0).

$$ \text{Volume State}_{i, t}=1, \text{Volume }_{i, t}>\text{ Volume }_{i, t-1} $$

$$ \text{Volume State}_{i, t}=0, \text{Volume }_{i, t} \leqslant \text{ Volume }_{i, t-1} $$

## 1.2.2 $\,$ Stock Price State Identification

The relative position of the stock price is characterized by the historical percentile of the current stock price over the past N days. When the closing price of stock i at time t is greater than or equal to the 90th percentile over the past N days, the current stock price is classified as Top. When it is greater than the 70th percentile, it is classified as High. When it is between the 30th and 70th percentiles, it is classified as Medium. When it is below the 30th percentile, it is classified as Low. When it is less than or equal to the 10th percentile, it is classified as Bottom.

$$
\begin{array}{l}
\text{Top}_{i, t} = 1,\quad \text{Close}_{i, t} \geqslant Q_{90\%, i, N} \\
\text{High}_{i, t} = 1,\quad \text{Close}_{i, t} > Q_{70\%, i, N} \\
\text{Medium}_{i, t} = 1,\quad Q_{30\%, i, N} \leqslant \text{Close}_{i, t} \leqslant Q_{70\%, i, N} \\
\text{Low}_{i, t} = 1,\quad \text{Close}_{i, t} < Q_{30\%, i, N} \\
\text{Bottom}_{i, t} = 1,\quad \text{Close}_{i, t} \leqslant Q_{10\%, i, N}
\end{array}
$$

Specifically, in the subsequent candlestick pattern effectiveness testing process, price data from the past 120 trading days (N=120) is used for stock price state identification.

In [8]:
# To prevent cases where pre_close equals 0, we need to exclude these instances
all_stock_daily = all_stock_daily.loc[idx[all_stock_daily['pre_close']!=0,:],:]
all_stock_daily = all_stock_daily.loc[idx[all_stock_daily['pre_close']!=0,:],:]

In [9]:
for stock in all_stocks:
  # Candlestick pattern recognition
    # Single stock market data
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # # Calculate amplitude
    all_stock_daily.loc[idx[:,stock],'amplitude'] = \
            (stock_daily['high'] - stock_daily['low']) / stock_daily['pre_close']

    # Calculate body ratio
    all_stock_daily.loc[idx[:,stock],'body'] = \
            (stock_daily['close'] - stock_daily['open']).abs() / stock_daily['pre_close'] / all_stock_daily.loc[idx[:,stock],'amplitude']

    # Calculate upper shadow ratio
    all_stock_daily.loc[idx[:,stock],'upper_shadow'] = \
            (stock_daily['high'] - pd.concat([stock_daily['open'], stock_daily['close']], axis=1).max(axis=1)) / stock_daily['pre_close'] / all_stock_daily.loc[idx[:,stock],'amplitude']

    # Calculate lower shadow ratio
    all_stock_daily.loc[idx[:,stock],'lower_shadow'] = \
            (pd.concat([stock_daily['open'], stock_daily['close']], axis=1).min(axis=1) - stock_daily['low']) / stock_daily['pre_close'] / all_stock_daily.loc[idx[:,stock],'amplitude']

    # Determine if it's a bullish candle
    all_stock_daily.loc[idx[:,stock],'is_bullish'] = (stock_daily['close'] > stock_daily['open']).astype(int)

  # Price-volume state identification
    # Volume state identification
    all_stock_daily.loc[idx[:,stock], 'volume_state'] = (
        stock_daily['vol'] > stock_daily['vol'].shift(1)
    ).astype(int)

    # Stock price state identification
    close_pct = stock_daily['close'].rolling(window=120, min_periods=1).rank(pct=True)
    
    all_stock_daily.loc[idx[:,stock], 'is_top'] = (close_pct >= 0.9).astype(int)                    # Top: ≥90%
    all_stock_daily.loc[idx[:,stock], 'is_high'] = (close_pct > 0.7).astype(int)                     # High: >70%
    all_stock_daily.loc[idx[:,stock], 'is_medium'] = ((close_pct >= 0.3) & (close_pct <= 0.7)).astype(int)  # Medium: 30%-70%
    all_stock_daily.loc[idx[:,stock], 'is_low'] = (close_pct < 0.3).astype(int)                      # Low: <30%
    all_stock_daily.loc[idx[:,stock], 'is_bottom'] = (close_pct <= 0.1).astype(int)                   # Bottom: ≤10%

  # 20-day average trading amount
    all_stock_daily.loc[idx[:,stock], 'avg_amount_20d'] = stock_daily['amount'].rolling(
        window=20, min_periods=1
    ).mean()

In [10]:
# Checking the indicators again shows many NaN values in body, upper_shadow, and lower_shadow columns
# Based on their calculation formulas, this is likely caused by amplitude equaling 0. 
# This occurs when a stock hits limit-up immediately at market open and stays there all day
# We need to further filter out these stocks
all_stock_daily.isna().sum()

open                  0
high                  0
low                   0
close                 0
pre_close             0
change                0
pct_chg               0
vol                   0
amount                0
amplitude             0
body              64310
upper_shadow      64312
lower_shadow      64310
is_bullish            0
volume_state          0
is_top                0
is_high               0
is_medium             0
is_low                0
is_bottom             0
avg_amount_20d        0
dtype: int64

In [11]:
all_stock_daily.dropna(inplace=True)

In [12]:
all_stock_daily

open   high    low  close  pre_close  change  pct_chg  \
trade_date ts_code                                                             
2014-01-02 000001.SZ   5.37   5.45   5.34   5.42       5.43   -0.01    -0.18   
           000002.SZ   5.14   5.19   5.09   5.14       5.16   -0.02    -0.39   
           000004.SZ  11.64  11.87  11.45  11.81      11.65    0.16     1.37   
           000006.SZ   3.82   3.83   3.75   3.77       3.82   -0.05    -1.31   
           000007.SZ   9.26   9.52   9.15   9.36       9.25    0.11     1.19   
...                     ...    ...    ...    ...        ...     ...      ...   
2025-09-26 920445.BJ  11.02  11.43  11.02  11.06      11.02    0.04     0.36   
           920489.BJ  23.46  23.76  22.54  22.57      23.72   -1.15    -4.85   
           920682.BJ   9.82  10.25   9.81  10.08       9.88    0.20     2.02   
           920799.BJ  55.86  55.92  54.50  54.50      55.22   -0.72    -1.30   
           920819.BJ   4.25   4.31   4.20   4.25       4.24    0.01     0.24   

                            vol       amount  amplitude  ...  upper_shadow  \
trade_date ts_code                                       ...                 
2014-01-02 000001.SZ  489910.89  596223.7444   0.020258  ...      0.272727   
           000002.SZ  485299.22  387391.0664   0.019380  ...      0.500000   
           000004.SZ    9714.74   11417.5556   0.036052  ...      0.142857   
           000006.SZ   88230.28   43051.9277   0.020942  ...      0.125000   
           000007.SZ   32811.75   46266.7890   0.040000  ...      0.432432   
...                         ...          ...        ...  ...           ...   
2025-09-26 920445.BJ   19097.22   21424.6980   0.037205  ...      0.902439   
           920489.BJ   73613.53  168578.2070   0.051433  ...      0.245902   
           920682.BJ   60706.91   61218.3150   0.044534  ...      0.386364   
           920799.BJ   23112.55  127756.1470   0.025715  ...      0.042254   
           920819.BJ   85665.17   36397.3400   0.025943  ...      0.545455   

                      lower_shadow  is_bullish  volume_state  is_top  is_high  \
trade_date ts_code                                                              
2014-01-02 000001.SZ      0.272727         1.0           0.0     1.0      1.0   
           000002.SZ      0.500000         0.0           0.0     1.0      1.0   
           000004.SZ      0.452381         1.0           0.0     1.0      1.0   
           000006.SZ      0.250000         0.0           0.0     1.0      1.0   
           000007.SZ      0.297297         1.0           0.0     1.0      1.0   
...                            ...         ...           ...     ...      ...   
2025-09-26 920445.BJ      0.000000         1.0           1.0     0.0      0.0   
           920489.BJ      0.024590         0.0           0.0     0.0      0.0   
           920682.BJ      0.022727         1.0           1.0     0.0      0.0   
           920799.BJ      0.000000         0.0           0.0     0.0      0.0   
           920819.BJ      0.454545         0.0           0.0     0.0      0.0   

                      is_medium  is_low  is_bottom  avg_amount_20d  
trade_date ts_code                                                  
2014-01-02 000001.SZ        0.0     0.0        0.0    596223.74440  
           000002.SZ        0.0     0.0        0.0    387391.06640  
           000004.SZ        0.0     0.0        0.0     11417.55560  
           000006.SZ        0.0     0.0        0.0     43051.92770  
           000007.SZ        0.0     0.0        0.0     46266.78900  
...                         ...     ...        ...             ...  
2025-09-26 920445.BJ        1.0     0.0        0.0     45104.28675  
           920489.BJ        0.0     1.0        0.0    162566.70455  
           920682.BJ        1.0     0.0        0.0     55766.14700  
           920799.BJ        0.0     1.0        1.0    206737.39420  
           920819.BJ        1.0     0.0        0.0     86165.64425  

[10547491 rows 

In [13]:
# Daily industry closing price data
all_stock_price_daily = all_stock_daily['close'].reset_index().pivot(
    index='trade_date',     
    columns='ts_code',  
    values='close'       
)

In [14]:
all_stock_price_daily

ts_code,000001.SZ,000002.SZ,000004.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,000012.SZ,...,920112.BJ,920116.BJ,920118.BJ,920128.BJ,920167.BJ,920445.BJ,920489.BJ,920682.BJ,920799.BJ,920819.BJ
trade_date,,,,,,,,,,,,,,,,,,,,,
2014-01-02,5.42,5.14,11.81,3.77,9.36,2.03,4.67,7.70,5.66,4.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03,5.29,5.04,11.80,3.66,9.33,2.02,4.54,7.76,5.53,4.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06,5.17,4.81,12.30,3.45,9.36,1.92,4.51,7.22,5.33,3.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07,5.15,4.78,12.23,3.45,9.72,1.90,4.50,7.24,5.33,3.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,5.21,4.77,12.23,3.40,9.74,1.86,4.42,7.38,5.22,3.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-22,11.38,7.00,10.56,7.92,7.05,2.86,10.21,4.47,9.02,4.67,...,40.18,70.72,27.82,31.68,19.92,11.51,23.69,9.95,57.88,4.45
2025-09-23,11.52,6.76,10.21,8.43,6.92,2.82,10.29,4.37,8.95,4.65,...,39.27,68.78,26.77,30.80,19.18,11.10,23.51,9.65,54.73,4.29
2025-09-24,11.46,6.84,10.65,9.27,6.97,2.85,11.32,4.37,9.14,4.71,...,39.53,69.25,27.32,31.02,19.66,11.30,24.53,9.84,55.28,4.33


In [15]:
def backtest_strategy(selected_stocks_series: pd.Series,
                     price_df: pd.DataFrame, 
                     holding_periods: List[int] = [5, 10, 20],
                     volume_price_state: Optional[str] = None) -> pd.DataFrame:
    """
    Complete backtesting function
    
    Parameters:
    -----------
    selected_stocks_series : pd.Series
        Daily stock selection list, indexed by trading date
    price_df : pd.DataFrame
        Market-wide daily closing price data, indexed by trading date with stock codes as columns
    holding_periods : List[int]
        List of holding periods, default [5, 10, 20] trading days
    volume_price_state : str, optional
        Price-volume state, if provided will add this column to results
    
    Returns:
    --------
    pd.DataFrame
        Backtesting results table
    """
    
    # 1. Data preprocessing
    selected_stocks_series.index = pd.to_datetime(selected_stocks_series.index)
    price_df.index = pd.to_datetime(price_df.index)
    
    # Align data indices
    common_dates = selected_stocks_series.index.intersection(price_df.index)
    selected_stocks_series = selected_stocks_series.loc[common_dates].sort_index()
    price_df = price_df.loc[common_dates].sort_index()
    
    results = []
    
    # 2. Calculate for each holding period
    for hold_days in holding_periods:
        all_returns = []
        trade_dates = selected_stocks_series.index
        
        # 3. Calculate returns for each trading day
        for i, current_date in enumerate(trade_dates):
            stock_list = selected_stocks_series.loc[current_date]
            
            # Check if stock list is valid
            if not isinstance(stock_list, list) or len(stock_list) == 0:
                continue
                
            # Find holding period end date
            future_index = i + hold_days
            if future_index >= len(trade_dates):
                continue
                
            end_date = trade_dates[future_index]
            
            try:
                # Get prices for current and end dates
                current_prices = price_df.loc[current_date, stock_list]
                future_prices = price_df.loc[end_date, stock_list]
                
                # Handle missing values - keep only stocks with data on both dates
                valid_mask = (~current_prices.isna()) & (~future_prices.isna())
                valid_stocks = current_prices.index[valid_mask]
                
                if len(valid_stocks) == 0:
                    continue
                    
                current_prices = current_prices[valid_stocks]
                future_prices = future_prices[valid_stocks]
                
                # Calculate returns for each stock
                returns = (future_prices - current_prices) / current_prices
                
                # Calculate equal-weighted portfolio return (average)
                portfolio_return = returns.mean()
                all_returns.append(portfolio_return)
                
            except (KeyError, TypeError):
                # Skip if stocks don't exist in price data or data type error
                continue
        
        if len(all_returns) == 0:
            continue
            
        returns_series = pd.Series(all_returns)
        
        # 4. Calculate metrics (format exactly matches the image)
        result_row = {
            'Holding Period': f'Hold {hold_days:02d} days',
            'Average Return': f"{returns_series.mean() * 100:.2f}%",
            'Return Std Dev': f"{returns_series.std():.2f}",
            'Max Gain': f"{returns_series.max() * 100:.2f}%",
            'Max Loss': f"{returns_series.min() * 100:.2f}%",
            'Median Return': f"{returns_series.median() * 100:.2f}%",
            'Win Rate': f"{(returns_series > 0).mean() * 100:.2f}%"
        }
        
        # 5. Handle price-volume state column
        if volume_price_state is not None:
            result_row = {'Price-Volume State': volume_price_state, **result_row}
        
        results.append(result_row)
    
    # 6. Create final results DataFrame
    result_df = pd.DataFrame(results)
    
    # 7. Adjust column order
    if not result_df.empty:
        if volume_price_state is not None and 'Price-Volume State' in result_df.columns:
            # Column order with price-volume state
            cols = ['Price-Volume State', 'Holding Period', 'Average Return', 'Return Std Dev',
                    'Max Gain', 'Max Loss', 'Median Return', 'Win Rate']
        else:
            # Column order without price-volume state
            cols = ['Holding Period', 'Average Return', 'Return Std Dev',
                    'Max Gain', 'Max Loss', 'Median Return', 'Win Rate']
        
        # Keep only existing columns
        existing_cols = [col for col in cols if col in result_df.columns]
        result_df = result_df[existing_cols]
    
    return result_df

# 2 $\,$ Candlestick Pattern Effectiveness Testing

Based on candlestick pattern recognition and price-volume state identification, we will utilize historical time-series price-volume information of all stocks to identify effective candlestick patterns and explore their compatibility with various price-volume states, aiming to further improve portfolio returns and prediction accuracy.

## 2.1 $\,$ Candlestick Pattern Backtesting Framework
Backtesting period: January 2, 2014 to September 26, 2025  
Stock universe: A-shares listed in Shanghai, Shenzhen, and Beijing exchanges  

Candlestick pattern effectiveness testing methodology:  
When a target candlestick pattern appears at time t, we calculate the following metrics for the qualifying stock portfolio from t+1 until the end of the holding period:  
• Average return  

• Return standard deviation  

• Maximum gain  

• Maximum loss  

• Median return  

• Win rate  

These metrics are used to evaluate the effectiveness of candlestick patterns.

Holding periods:  
• 5-day holding period  

• 10-day holding period  

• 20-day holding period  

Special treatment:  
Exclude stocks with zero average trading volume over the past 20 days.

# 3 $\,$ Effective Candlestick Patterns

## 3.1 $\,$ Single-Candle Patterns

### 3.1.1 $\,$ Large Bullish Candle Pattern
The large bullish candle is a positive candlestick pattern characterized by a significantly higher closing price than the opening price, with the real body occupying a relatively large proportion of the total range.

Key features:

1) The current candle must be bullish (close > open);

2) Both upper shadow ratio (Upper Shadow) and lower shadow ratio (Lower Shadow) are less than 5%;

3) Daily return is greater than or equal to 1%;

4) The opening price (Open) falls between the previous day's high and low prices.

<div align="center">
<img src="./img/2.png" width="800">
</div>

In [16]:
# Large Bullish Candle Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Current candle is bullish (close > open) 
    condition_1 = (stock_daily['is_bullish'] == 1)
    # Condition 2: Both upper shadow and lower shadow ratios are less than 5%
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    # Condition 3: Daily return is greater than or equal to 1%
    condition_3 = stock_daily['pct_chg'] >= 1
    # Condition 4: Opening price is between previous day's high and low
    condition_4 = (stock_daily['open'] >= stock_daily['low'].shift(1)) & (stock_daily['open'] <= stock_daily['high'].shift(1))

    condition = condition_1 & condition_2 & condition_3 & condition_4

    all_stock_daily.loc[idx[:,stock],'long_white_candle'] = condition.astype(int)

# Filter all large bullish candle records (long_white_candle == 1)
long_white_records = all_stock_daily[all_stock_daily['long_white_candle'] == 1]

# Group by date and collect stock codes for each day
daily_long_white_stocks = long_white_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [17]:
# Large Bullish Candle Pattern Effectiveness Test
result_long_white = backtest_strategy(daily_long_white_stocks, all_stock_price_daily)
result_long_white

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.48%,0.05,40.82%,-35.03%,0.60%,56.05%
1,Hold 10 days,0.41%,0.07,56.77%,-51.86%,0.51%,53.91%
2,Hold 20 days,0.50%,0.10,85.44%,-49.10%,0.36%,52.02%


In [18]:
# Combining Large Bullish Candle with Price-Volume States

# Combine with Volume Expansion
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Current candle is bullish (close > open) 
    condition_1 = (stock_daily['is_bullish'] == 1)
    # Condition 2: Both upper and lower shadow ratios < 5%
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    # Condition 3: Daily return >= 1%
    condition_3 = (stock_daily['pct_chg'] >= 1)
    # Condition 4: Open price between previous day's low and high
    condition_4 = (stock_daily['open'] >= stock_daily['low'].shift(1)) & (stock_daily['open'] <= stock_daily['high'].shift(1))
    # Condition 5: Volume expansion
    condition_5 = (stock_daily['volume_state'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5

    all_stock_daily.loc[idx[:,stock],'long_white_candle_and_volume_expansion​'] = condition.astype(int)

# Filter all Large Bullish Candle + Volume Expansion records
long_white_and_volume_expansion_records = all_stock_daily[all_stock_daily['long_white_candle_and_volume_expansion​'] == 1]
# Group by date and collect stock codes
daily_long_white_and_volume_expansion_stocks = long_white_and_volume_expansion_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with Low Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Current candle is bullish 
    condition_1 = (stock_daily['is_bullish'] == 1)
    # Condition 2: Both shadows < 5%
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    # Condition 3: Daily return >= 1%
    condition_3 = stock_daily['pct_chg'] >= 1
    # Condition 4: Open price between previous day's range
    condition_4 = (stock_daily['open'] >= stock_daily['low'].shift(1)) & (stock_daily['open'] <= stock_daily['high'].shift(1))
    # Condition 5: Low price level
    condition_5 = (stock_daily['is_low'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5

    all_stock_daily.loc[idx[:,stock],'long_white_candle_and_low_level'] = condition.astype(int)
# Filter all Large Bullish Candle + Low Level records
long_white_and_low_level_records = all_stock_daily[all_stock_daily['long_white_candle_and_low_level'] == 1]
# Group by date and collect stock codes
daily_long_white_and_low_level_stocks = long_white_and_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with Volume Expansion + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Current candle is bullish 
    condition_1 = (stock_daily['is_bullish'] == 1)
    # Condition 2: Both shadows < 5%
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    # Condition 3: Daily return >= 1%
    condition_3 = stock_daily['pct_chg'] >= 1
    # Condition 4: Open price between previous day's range
    condition_4 = (stock_daily['open'] >= stock_daily['low'].shift(1)) & (stock_daily['open'] <= stock_daily['high'].shift(1))
    # Condition 5: Volume expansion + Low level
    condition_5 = (stock_daily['volume_state'] == 1) & (stock_daily['is_low'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5

    all_stock_daily.loc[idx[:,stock],'long_white_candle_and_volume_expansion_and_low_level'] = condition.astype(int)
# Filter all Large Bullish Candle + Volume Expansion + Low Level records
long_white_and_volume_expansion_and_low_level_records = all_stock_daily[all_stock_daily['long_white_candle_and_volume_expansion_and_low_level'] == 1]
# Group by date and collect stock codes
daily_long_white_and_volume_expansion_and_low_level_stocks = long_white_and_volume_expansion_and_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [19]:
# Large Bullish Candle + Volume Expansion Pattern Effectiveness Test
result_long_white_and_volume_expansion = backtest_strategy(daily_long_white_and_volume_expansion_stocks, all_stock_price_daily, volume_price_state="Volume Expansion")
result_long_white_and_volume_expansion

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion,Hold 05 days,0.46%,0.05,103.93%,-35.92%,0.51%,54.74%
1,Volume Expansion,Hold 10 days,0.31%,0.07,56.77%,-52.45%,0.44%,53.37%
2,Volume Expansion,Hold 20 days,0.33%,0.10,85.44%,-49.10%,0.07%,50.41%


In [20]:
# Large Bullish Candle + Low Price Level Pattern Effectiveness Test
result_long_white_and_low_level = backtest_strategy(daily_long_white_and_low_level_stocks, all_stock_price_daily, volume_price_state="Low Price Level")
result_long_white_and_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Price Level,Hold 05 days,0.88%,0.09,150.39%,-42.84%,0.56%,54.55%
1,Low Price Level,Hold 10 days,1.58%,0.14,266.48%,-41.20%,0.65%,53.60%
2,Low Price Level,Hold 20 days,2.11%,0.16,183.95%,-41.96%,0.05%,50.23%


In [21]:
# Large Bullish Candle + Volume Expansion + Low Price Level Pattern Effectiveness Test
result_long_white_and_volume_expansion_and_low_level = backtest_strategy(daily_long_white_and_volume_expansion_and_low_level_stocks, all_stock_price_daily, volume_price_state="Volume Expansion + Low Level")
result_long_white_and_volume_expansion_and_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + Low Level,Hold 05 days,0.75%,0.08,87.40%,-48.45%,0.41%,53.13%
1,Volume Expansion + Low Level,Hold 10 days,1.23%,0.12,122.45%,-41.20%,0.47%,53.00%
2,Volume Expansion + Low Level,Hold 20 days,1.91%,0.16,236.83%,-47.02%,0.03%,50.06%


### 3.1.2 $\,$ Large Bearish Candle Pattern
The large bearish candle is a common candlestick pattern characterized by a significantly higher opening price than the closing price, with the real body occupying a relatively large proportion of the total range. While simple bearish candles show weak effectiveness, ​gap-down bearish candles​ demonstrate stronger predictive power.

Key Features:

1) ​Bearish Candle: Close < Open (filled/green body)

2) ​Minimal Shadows: Both upper and lower shadow ratios < 5%

3) ​Price Decline: Daily return ≤ -1%

4) Gap-Down Open: Opening price < Previous day's low (stronger signal)

<div align="center">
<img src="./img/3.png" width="800">
</div>

In [22]:
# Large Bearish Candle Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Current candle is bearish (close < open)
    condition_1 = (stock_daily['is_bullish'] == 0)
    # Condition 2: Both upper shadow and lower shadow ratios are less than 5%
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    # Condition 3: Daily return is less than or equal to -1%
    condition_3 = stock_daily['pct_chg'] <= -1
    # Condition 4: Opening price is below previous day's low (gap down)
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1))

    condition = condition_1 & condition_2 & condition_3 & condition_4

    all_stock_daily.loc[idx[:,stock],'long_black_candle'] = condition.astype(int)

# Filter all large bearish candle records
long_black_records = all_stock_daily[all_stock_daily['long_black_candle'] == 1]

# Group by date and collect stock codes for each day
daily_long_black_stocks = long_black_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [23]:
daily_long_black_stocks

trade_date
2014-01-06    [000401.SZ, 000663.SZ, 000829.SZ, 000969.SZ, 0...
2014-01-09                                          [002506.SZ]
2014-01-10    [000004.SZ, 000301.SZ, 000756.SZ, 000825.SZ, 0...
2014-01-13                                          [000598.SZ]
2014-01-16                               [002322.SZ, 600072.SH]
                                    ...                        
2025-09-16                                          [000609.SZ]
2025-09-19                                          [002269.SZ]
2025-09-23                               [002620.SZ, 600749.SH]
2025-09-24                                          [301038.SZ]
2025-09-26    [000036.SZ, 000409.SZ, 000810.SZ, 001208.SZ, 0...
Length: 1822, dtype: object

In [24]:
# Gap-Down Bearish Candle Pattern Effectiveness Test
result_long_black = backtest_strategy(daily_long_black_stocks, all_stock_price_daily)
result_long_black

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,-0.98%,0.10,76.82%,-53.32%,-1.67%,39.21%
1,Hold 10 days,-0.12%,0.14,140.92%,-59.50%,-1.28%,43.57%
2,Hold 20 days,0.98%,0.18,150.76%,-50.37%,-1.64%,44.40%


In [25]:
# Combine with Low Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    condition_3 = stock_daily['pct_chg'] <= -1
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1))
    condition_5 = (stock_daily['is_low'] == 1)
    all_stock_daily.loc[idx[:,stock],'long_black_candle_and_low_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5).astype(int)

# Combine with High Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    condition_3 = stock_daily['pct_chg'] <= -1
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1))
    condition_5 = (stock_daily['is_high'] == 1)
    all_stock_daily.loc[idx[:,stock],'long_black_candle_and_high_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5).astype(int)

# Combine with Volume Contraction + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    condition_3 = stock_daily['pct_chg'] <= -1
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1))
    condition_5 = (stock_daily['volume_state'] == 0) & (stock_daily['is_high'] == 0)
    all_stock_daily.loc[idx[:,stock],'long_black_candle_and_volume_contraction_and_low_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5).astype(int)

# Combine with Volume Expansion + High Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['lower_shadow'] < 0.05)
    condition_3 = stock_daily['pct_chg'] <= -1
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1))
    condition_5 = (stock_daily['volume_state'] == 1) & (stock_daily['is_high'] == 1)
    all_stock_daily.loc[idx[:,stock],'long_black_candle_and_volume_expansion_and_high_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5).astype(int)

# Filter records and group by date
long_black_and_low_level_records = all_stock_daily[all_stock_daily['long_black_candle_and_low_level'] == 1]
daily_long_black_and_low_level_stocks = long_black_and_low_level_records.groupby(level='trade_date').apply(lambda x: x.index.get_level_values('ts_code').tolist())

long_black_and_high_level_records = all_stock_daily[all_stock_daily['long_black_candle_and_high_level'] == 1]
daily_long_black_and_high_level_stocks = long_black_and_high_level_records.groupby(level='trade_date').apply(lambda x: x.index.get_level_values('ts_code').tolist())

long_black_and_volume_contraction_and_low_level_records = all_stock_daily[all_stock_daily['long_black_candle_and_volume_contraction_and_low_level'] == 1]
daily_long_black_and_volume_contraction_and_low_level_stocks = long_black_and_volume_contraction_and_low_level_records.groupby(level='trade_date').apply(lambda x: x.index.get_level_values('ts_code').tolist())

long_black_and_volume_expansion_and_high_level_records = all_stock_daily[all_stock_daily['long_black_candle_and_volume_expansion_and_high_level'] == 1]
daily_long_black_and_volume_expansion_and_high_level_stocks = long_black_and_volume_expansion_and_high_level_records.groupby(level='trade_date').apply(lambda x: x.index.get_level_values('ts_code').tolist())

In [26]:
# Gap-Down Bearish Candle + Low Price Level Pattern Effectiveness Test
result_long_black_and_low_level = backtest_strategy(daily_long_black_and_low_level_stocks, all_stock_price_daily,volume_price_state="Low Price Level")
result_long_black_and_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Price Level,Hold 05 days,-0.39%,0.13,139.19%,-60.69%,-1.57%,39.70%
1,Low Price Level,Hold 10 days,0.51%,0.16,154.87%,-46.70%,-1.79%,42.44%
2,Low Price Level,Hold 20 days,1.69%,0.25,377.02%,-53.26%,-1.98%,44.72%


In [27]:
# Gap-Down Bearish Candle + High Price Level Pattern Effectiveness Test
result_long_black_and_high_level = backtest_strategy(daily_long_black_and_high_level_stocks, all_stock_price_daily,volume_price_state="High Price Level")
result_long_black_and_high_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,High Price Level,Hold 05 days,-1.08%,0.16,158.44%,-46.39%,-2.24%,41.00%
1,High Price Level,Hold 10 days,0.18%,0.22,189.05%,-47.66%,-2.85%,41.28%
2,High Price Level,Hold 20 days,2.59%,0.34,382.05%,-57.49%,-3.24%,41.26%


In [28]:
# Gap-Down Bearish Candle + Volume Contraction + Low Price Level Pattern Effectiveness Test
result_long_black_and_volume_contraction_and_low_level = backtest_strategy(daily_long_black_and_volume_contraction_and_low_level_stocks, all_stock_price_daily,volume_price_state="Volume Contraction + Low Level")
result_long_black_and_volume_contraction_and_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Contraction + Low Level,Hold 05 days,-0.35%,0.16,152.73%,-43.50%,-2.31%,38.77%
1,Volume Contraction + Low Level,Hold 10 days,1.24%,0.20,152.85%,-40.77%,-1.97%,42.79%
2,Volume Contraction + Low Level,Hold 20 days,2.17%,0.30,341.98%,-67.80%,-2.44%,44.56%


In [29]:
# Gap-Down Bearish Candle + Volume Expansion + High Price Level Pattern Effectiveness Test
result_long_black_and_volume_expansion_and_high_level = backtest_strategy(daily_long_black_and_volume_expansion_and_high_level_stocks, all_stock_price_daily,volume_price_state="Volume Expansion + High Level")
result_long_black_and_volume_expansion_and_high_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + High Level,Hold 05 days,1.12%,0.19,148.85%,-49.18%,-1.64%,43.60%
1,Volume Expansion + High Level,Hold 10 days,1.94%,0.24,169.17%,-58.32%,-1.54%,46.27%
2,Volume Expansion + High Level,Hold 20 days,6.41%,0.48,668.71%,-66.21%,-1.81%,46.64%


### 3.1.3 $\,$ Hammer Candlestick Pattern
The hammer is a common candlestick pattern named for its resemblance to a hammer. It is characterized by a small real body, a long lower shadow, and a very small or non-existent upper shadow. The pattern can be either bullish or bearish.

Key Features:

1) ​Upper Shadow Ratio​ < 5%

2) Body Ratio​ between 5% and 30%

<div align="center">
<img src="./img/4.png" width="800">
</div>

In [30]:
# Hammer Candlestick Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Upper shadow ratio < 5% 
    condition_1 = (stock_daily['upper_shadow'] < 0.05)
    # Condition 2: Body ratio between 5% and 30%
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)

    condition = condition_1 & condition_2

    all_stock_daily.loc[idx[:,stock],'hammer'] = condition.astype(int)

# Filter all hammer pattern records
hammer_records = all_stock_daily[all_stock_daily['hammer'] == 1]

# Group by date and collect stock codes
daily_hammer_stocks = hammer_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [31]:
# Hammer Candlestick Pattern Effectiveness Test
result_hammer = backtest_strategy(daily_hammer_stocks, all_stock_price_daily)
result_hammer

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,-0.02%,0.05,61.09%,-33.35%,0.05%,50.55%
1,Hold 10 days,0.21%,0.06,54.62%,-38.25%,0.31%,52.54%
2,Hold 20 days,0.79%,0.10,153.68%,-48.06%,0.47%,52.61%


In [32]:
# Hammer Pattern Combined with Price-Volume States

# Combine with Gap Down
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['upper_shadow'] < 0.05)
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)
    condition_3 = (stock_daily['open'] < stock_daily['low'].shift(1))
    all_stock_daily.loc[idx[:,stock],'hammer_and_gap_down'] = (condition_1 & condition_2 & condition_3).astype(int)

hammer_and_gap_down_records = all_stock_daily[all_stock_daily['hammer_and_gap_down'] == 1]
daily_hammer_and_gap_down_stocks = hammer_and_gap_down_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with High Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['upper_shadow'] < 0.05)
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)
    condition_3 = (stock_daily['is_high'] == 1)
    all_stock_daily.loc[idx[:,stock],'hammer_and_high_level'] = (condition_1 & condition_2 & condition_3).astype(int)

hammer_and_high_level_records = all_stock_daily[all_stock_daily['hammer_and_high_level'] == 1]
daily_hammer_and_high_level_stocks = hammer_and_high_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with Gap Down + High Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['upper_shadow'] < 0.05)
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)
    condition_3 = (stock_daily['open'] < stock_daily['low'].shift(1)) & (stock_daily['is_high'] == 1)
    all_stock_daily.loc[idx[:,stock],'hammer_and_gap_down_and_high_level'] = (condition_1 & condition_2 & condition_3).astype(int)

hammer_and_gap_down_and_high_level_records = all_stock_daily[all_stock_daily['hammer_and_gap_down_and_high_level'] == 1]
daily_hammer_and_gap_down_and_high_level_stocks = hammer_and_gap_down_and_high_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [33]:
# Hammer + Gap Down Performance
result_hammer_and_gap_down = backtest_strategy(daily_hammer_and_gap_down_stocks, all_stock_price_daily, volume_price_state="Gap Down")
result_hammer_and_gap_down

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Gap Down,Hold 05 days,0.15%,0.09,68.75%,-50.93%,-0.27%,48.17%
1,Gap Down,Hold 10 days,0.54%,0.13,78.12%,-56.23%,-0.99%,44.82%
2,Gap Down,Hold 20 days,1.39%,0.19,151.62%,-54.21%,-1.37%,46.41%


In [34]:
# Hammer + High Level Performance
result_hammer_and_high_level = backtest_strategy(daily_hammer_and_high_level_stocks, all_stock_price_daily, volume_price_state="High Level")
result_hammer_and_high_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,High Level,Hold 05 days,-0.44%,0.06,77.18%,-54.76%,-0.19%,47.84%
1,High Level,Hold 10 days,-0.17%,0.09,106.85%,-59.89%,-0.11%,49.19%
2,High Level,Hold 20 days,0.12%,0.12,213.68%,-56.66%,-0.23%,48.75%


In [36]:
# Hammer + Gap Down + High Level Performance
result_hammer_and_gap_down_and_high_level = backtest_strategy(daily_hammer_and_gap_down_and_high_level_stocks, all_stock_price_daily, volume_price_state="Gap Down + High Level")
result_hammer_and_gap_down_and_high_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Gap Down + High Level,Hold 05 days,0.21%,0.15,124.75%,-45.48%,-0.84%,46.65%
1,Gap Down + High Level,Hold 10 days,1.49%,0.22,179.41%,-56.31%,-1.05%,47.92%
2,Gap Down + High Level,Hold 20 days,4.05%,0.32,350.58%,-73.98%,-0.89%,48.17%


### 3.1.4 $\,$ Inverted Hammer Candlestick Pattern
The inverted hammer, resembling an upside-down hammer, is typically considered a potential bottom reversal pattern. It features a small real body, a long upper shadow, and a very small or non-existent lower shadow. The pattern can be either bullish or bearish.

Key Features:

1) Lower Shadow Ratio​ < 5%

2) Body Ratio​ between 5% and 30%

<div align="center">
<img src="./img/5.png" width="800">
</div>
<div align="center">
<img src="./img/6.png" width="800">
</div>

In [37]:
# Inverted Hammer Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Lower shadow ratio < 5%
    condition_1 = (stock_daily['lower_shadow'] < 0.05)
    # Condition 2: Body ratio between 5% and 30%
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)

    condition = condition_1 & condition_2

    all_stock_daily.loc[idx[:,stock],'inverted_hammer'] = condition.astype(int)

# Filter all inverted hammer records
inverted_hammer_records = all_stock_daily[all_stock_daily['inverted_hammer'] == 1]

# Group by date and collect stock codes
daily_inverted_hammer_stocks = inverted_hammer_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [38]:
# Inverted Hammer Pattern Effectiveness Test
result_inverted_hammer = backtest_strategy(daily_inverted_hammer_stocks, all_stock_price_daily)
result_inverted_hammer

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.17%,0.05,77.01%,-44.16%,0.30%,53.97%
1,Hold 10 days,0.39%,0.07,78.45%,-53.63%,0.46%,53.94%
2,Hold 20 days,1.11%,0.11,103.25%,-52.96%,0.62%,53.35%


In [39]:
# Combine inverted hammer with bottom price level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['lower_shadow'] < 0.05)
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)
    condition_3 = (stock_daily['is_bottom'] == 1)
    all_stock_daily.loc[idx[:,stock],'inverted_hammer_and_bottom_level'] = (condition_1 & condition_2 & condition_3).astype(int)

inverted_hammer_and_bottom_level_records = all_stock_daily[all_stock_daily['inverted_hammer_and_bottom_level'] == 1]
daily_inverted_hammer_and_bottom_level_stocks = inverted_hammer_and_bottom_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with bottom level + gap
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['lower_shadow'] < 0.05)
    condition_2 = (stock_daily['body'] >= 0.05) & (stock_daily['body'] <= 0.3)
    condition_3 = (stock_daily['is_bottom'] == 1)
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1)) | (stock_daily['open'] > stock_daily['high'].shift(1))
    all_stock_daily.loc[idx[:,stock],'inverted_hammer_and_bottom_level_and_gap'] = (condition_1 & condition_2 & condition_3 & condition_4).astype(int)

inverted_hammer_and_bottom_level_and_gap_records = all_stock_daily[all_stock_daily['inverted_hammer_and_bottom_level_and_gap'] == 1]
daily_inverted_hammer_and_bottom_level_and_gap_stocks = inverted_hammer_and_bottom_level_and_gap_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [40]:
# Inverted Hammer + Bottom Level Pattern Effectiveness Test
result_inverted_hammer_and_bottom_level = backtest_strategy(daily_inverted_hammer_and_bottom_level_stocks, all_stock_price_daily, volume_price_state="Bottom Level")
result_inverted_hammer

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.17%,0.05,77.01%,-44.16%,0.30%,53.97%
1,Hold 10 days,0.39%,0.07,78.45%,-53.63%,0.46%,53.94%
2,Hold 20 days,1.11%,0.11,103.25%,-52.96%,0.62%,53.35%


In [41]:
# Inverted Hammer + Bottom Level + Gap Pattern Effectiveness Test
result_inverted_hammer_and_bottom_level_and_gap = backtest_strategy(daily_inverted_hammer_and_bottom_level_and_gap_stocks, all_stock_price_daily, volume_price_state="Bottom Level + Gap")
result_inverted_hammer_and_bottom_level_and_gap

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Bottom Level + Gap,Hold 05 days,0.42%,0.14,239.13%,-42.89%,-0.65%,44.95%
1,Bottom Level + Gap,Hold 10 days,0.67%,0.16,249.23%,-47.83%,-1.04%,45.03%
2,Bottom Level + Gap,Hold 20 days,2.08%,0.22,297.51%,-50.56%,-1.32%,44.08%


### 3.1.5 $\,$ Doji Star Pattern
The Doji Star is an iconic candlestick pattern representing market equilibrium between bulls and bears. It features nearly identical opening and closing prices, with upper and lower shadows that may be similar or significantly different in length. Based on the relative proportions of upper and lower shadows, Doji Stars can be classified as Upper Doji, Long-Legged Doji, and Lower Doji.

**Upper Doji Characteristic Details:​​**

1) Body Ratio < 5%

2) Upper Shadow Ratio > 5%

3) Difference between Lower Shadow and Upper Shadow > 10%

**Long-Legged Doji Characteristic Details:​​**

1) Body Ratio < 5%

2) Absolute difference between Upper Shadow and Lower Shadow ≤ 10%

**Lower Doji Characteristic Details:​​**

1) Body Ratio < 5%

2) Lower Shadow Ratio > 5%

3) Difference between Upper Shadow and Lower Shadow > 10%

<div align="center">
<img src="./img/7.png" width="800">
</div>

In [42]:
# Upper Doji Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Body ratio < 5%
    condition_1 = (stock_daily['body'] < 0.05)
    # Condition 2: Upper shadow ratio > 5%
    condition_2 = stock_daily['upper_shadow'] > 0.05
    # Condition 3: Lower shadow minus upper shadow > 10%
    condition_3 = (stock_daily['lower_shadow'] - stock_daily['upper_shadow'] > 0.1)

    condition = condition_1 & condition_2 & condition_3

    all_stock_daily.loc[idx[:,stock],'upper_doji'] = condition.astype(int)

# Filter all upper doji records
upper_doji_records = all_stock_daily[all_stock_daily['upper_doji'] == 1]

# Group by date and collect stock codes
daily_upper_doji_stocks = upper_doji_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [44]:
# Upper Doji Pattern Effectiveness Test
result_dragonfly_doji = backtest_strategy(daily_upper_doji_stocks, all_stock_price_daily)
result_dragonfly_doji

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.12%,0.04,30.71%,-30.41%,0.24%,53.63%
1,Hold 10 days,0.41%,0.06,37.97%,-44.32%,0.45%,54.02%
2,Hold 20 days,1.07%,0.09,87.51%,-49.27%,0.94%,55.84%


In [45]:
# Long-Legged Doji Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Body ratio < 5%
    condition_1 = (stock_daily['body'] < 0.05)
    # Condition 2: Absolute difference between upper and lower shadows <= 10%
    condition_2 = (stock_daily['lower_shadow'] - stock_daily['upper_shadow']).abs() <= 0.1

    condition = condition_1 & condition_2

    all_stock_daily.loc[idx[:,stock],'long_legged_doji'] = condition.astype(int)

# Filter all long-legged doji records
long_legged_doji_records = all_stock_daily[all_stock_daily['long_legged_doji'] == 1]

# Group by date and collect stock codes
daily_long_legged_doji_stocks = long_legged_doji_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [46]:
# Long-Legged Doji Pattern Effectiveness Test
result_long_legged_doji = backtest_strategy(daily_long_legged_doji_stocks, all_stock_price_daily)
result_long_legged_doji

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.23%,0.05,44.37%,-44.48%,0.35%,54.86%
1,Hold 10 days,0.56%,0.06,46.86%,-44.88%,0.63%,56.67%
2,Hold 20 days,1.21%,0.09,98.63%,-53.44%,0.94%,56.16%


In [47]:
# Lower Doji Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Body ratio < 5%
    condition_1 = (stock_daily['body'] < 0.05)
    # Condition 2: Lower shadow ratio > 5%
    condition_2 = stock_daily['lower_shadow'] > 0.05
    # Condition 3: Upper shadow minus lower shadow > 10%
    condition_3 = (stock_daily['upper_shadow'] - stock_daily['lower_shadow']) > 0.1

    condition = condition_1 & condition_2 & condition_3

    all_stock_daily.loc[idx[:,stock],'lower_doji'] = condition.astype(int)

# Filter all lower doji records
lower_doji_records = all_stock_daily[all_stock_daily['lower_doji'] == 1]

# Group by date and collect stock codes
daily_lower_doji_stocks = lower_doji_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [49]:
# Lower Doji Pattern Effectiveness Test
result_gravestone_doji = backtest_strategy(daily_lower_doji_stocks, all_stock_price_daily)
result_gravestone_doji

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.23%,0.04,27.66%,-32.15%,0.37%,55.28%
1,Hold 10 days,0.51%,0.06,38.45%,-54.13%,0.61%,55.54%
2,Hold 20 days,1.15%,0.09,66.67%,-60.23%,0.96%,56.00%


In [50]:
# Upper Doji + Gap Down + Top Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Body ratio < 5%
    condition_1 = (stock_daily['body'] < 0.05)
    # Condition 2: Upper shadow ratio > 5%
    condition_2 = stock_daily['upper_shadow'] > 0.05
    # Condition 3: Lower shadow minus upper shadow > 10%
    condition_3 = (stock_daily['lower_shadow'] - stock_daily['upper_shadow'] > 0.1)
    # Condition 4: Gap down + Top price level
    condition_4 = (stock_daily['open'] < stock_daily['low'].shift(1)) & (stock_daily['is_top'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4

    all_stock_daily.loc[idx[:,stock],'upper_doji_gap_down_top'] = condition.astype(int)

# Filter all records matching the pattern
upper_doji_gap_down_top_records = all_stock_daily[all_stock_daily['upper_doji_gap_down_top'] == 1]

# Group by date and collect stock codes
daily_upper_doji_gap_down_top_stocks = upper_doji_gap_down_top_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [51]:
# Upper Doji + Gap Down + Top Level Pattern Effectiveness Test
result_dragonfly_doji_and_gap_down_and_top_level = backtest_strategy(daily_upper_doji_gap_down_top_stocks, all_stock_price_daily)
result_dragonfly_doji_and_gap_down_and_top_level

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,-0.13%,0.13,75.33%,-46.07%,-1.36%,43.36%
1,Hold 10 days,0.22%,0.18,82.48%,-62.39%,-2.10%,44.40%
2,Hold 20 days,0.47%,0.24,163.04%,-64.16%,-2.50%,43.37%


In [52]:
# Lower Doji + Gap Up + Bottom Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    # Condition 1: Body ratio < 5%
    condition_1 = (stock_daily['body'] < 0.05)
    # Condition 2: Lower shadow ratio > 5%
    condition_2 = stock_daily['lower_shadow'] > 0.05
    # Condition 3: Upper shadow minus lower shadow > 10%
    condition_3 = (stock_daily['upper_shadow'] - stock_daily['lower_shadow']) > 0.1
    # Condition 4: Gap up + Bottom price level
    condition_4 = (stock_daily['open'] > stock_daily['high'].shift(1)) & (stock_daily['is_bottom'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4

    all_stock_daily.loc[idx[:,stock],'lower_doji_gap_up_bottom'] = condition.astype(int)

# Filter all records matching the pattern
lower_doji_gap_up_bottom_records = all_stock_daily[all_stock_daily['lower_doji_gap_up_bottom'] == 1]

# Group by date and collect stock codes
daily_lower_doji_gap_up_bottom_stocks = lower_doji_gap_up_bottom_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [53]:
# Lower Doji + Gap Up + Bottom Level Pattern Effectiveness Test
result_gravestone_doji_gap_up_bottom_level = backtest_strategy(daily_lower_doji_gap_up_bottom_stocks, all_stock_price_daily)
result_gravestone_doji_gap_up_bottom_level

,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Hold 05 days,0.67%,0.15,123.55%,-63.79%,-0.51%,47.15%
1,Hold 10 days,1.49%,0.19,154.28%,-67.05%,-0.81%,47.30%
2,Hold 20 days,3.36%,0.30,354.61%,-65.71%,-2.10%,44.27%


## 3.2 Double Candlestick Patterns

### 3.2.1 Probe Line Pattern

The Probe Line pattern is a distinctive candlestick formation that reflects institutional testing and position consolidation. It typically appears during the mid-stage of market bottoms, the initial phase of uptrends, or during consolidation periods, indicating institutional attempts to test resistance levels and flush out weak holders.

Feature Details:

1) Both daily candlesticks are bullish (Yang);

2) Current day's upper shadow ratio (Upper Shadow) ≥ 50%;

3) Current day's lower shadow ratio (Lower Shadow) < 5%;

4) Previous day's upper shadow ratio (Upper Shadow) < 25%;

5) Previous day's lower shadow ratio (Lower Shadow) < 25%.

<div align="center">
<img src="./img/8.png" width="800">
</div>

In [56]:
# Probe Line Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Both days are bullish (Yang)
    condition_1 = (stock_daily['is_bullish'] == 1) & (stock_daily['is_bullish'].shift(1) == 1)
    
    # Condition 2: Current day upper shadow ratio >= 50%
    condition_2 = stock_daily['upper_shadow'] >= 0.5
    
    # Condition 3: Current day lower shadow ratio < 5%
    condition_3 = stock_daily['lower_shadow'] < 0.05
    
    # Condition 4: Previous day upper shadow ratio < 25%
    condition_4 = stock_daily['upper_shadow'].shift(1) < 0.25
    
    # Condition 5: Previous day lower shadow ratio < 25%
    condition_5 = stock_daily['lower_shadow'].shift(1) < 0.25

    # Combine all conditions
    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5

    # Mark the pattern
    all_stock_daily.loc[idx[:,stock],'probe_line'] = condition.astype(int)

# Filter all probe line records
probe_line_records = all_stock_daily[all_stock_daily['probe_line'] == 1]

# Group by date and collect stock codes
daily_probe_line_stocks = probe_line_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [57]:
# Probe Line Pattern Performance
result_exploratory_line = backtest_strategy(daily_probe_line_stocks, all_stock_price_daily, volume_price_state="Probe Line")
result_exploratory_line

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Probe Line,Hold 05 days,0.43%,0.06,55.54%,-44.11%,0.37%,53.60%
1,Probe Line,Hold 10 days,0.64%,0.09,69.74%,-58.68%,0.43%,53.34%
2,Probe Line,Hold 20 days,1.04%,0.12,83.15%,-46.85%,0.77%,53.14%


In [58]:
# Probe Line Pattern with Volume Expansion
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 1) & (stock_daily['is_bullish'].shift(1) == 1)
    condition_2 = stock_daily['upper_shadow'] >= 0.5
    condition_3 = stock_daily['lower_shadow'] < 0.05
    condition_4 = stock_daily['upper_shadow'].shift(1) < 0.25
    condition_5 = stock_daily['lower_shadow'].shift(1) < 0.25
    condition_6 = (stock_daily['volume_state'] == 1)
    all_stock_daily.loc[idx[:,stock],'probe_line_volume_expansion'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6).astype(int)

probe_line_volume_expansion_records = all_stock_daily[all_stock_daily['probe_line_volume_expansion'] == 1]
daily_probe_line_volume_expansion_stocks = probe_line_volume_expansion_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Probe Line with Low Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 1) & (stock_daily['is_bullish'].shift(1) == 1)
    condition_2 = stock_daily['upper_shadow'] >= 0.5
    condition_3 = stock_daily['lower_shadow'] < 0.05
    condition_4 = stock_daily['upper_shadow'].shift(1) < 0.25
    condition_5 = stock_daily['lower_shadow'].shift(1) < 0.25
    condition_6 = (stock_daily['is_low'] == 1)
    all_stock_daily.loc[idx[:,stock],'probe_line_low_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6).astype(int)

probe_line_low_level_records = all_stock_daily[all_stock_daily['probe_line_low_level'] == 1]
daily_probe_line_low_level_stocks = probe_line_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Probe Line with Volume Expansion + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 1) & (stock_daily['is_bullish'].shift(1) == 1)
    condition_2 = stock_daily['upper_shadow'] >= 0.5
    condition_3 = stock_daily['lower_shadow'] < 0.05
    condition_4 = stock_daily['upper_shadow'].shift(1) < 0.25
    condition_5 = stock_daily['lower_shadow'].shift(1) < 0.25
    condition_6 = (stock_daily['volume_state'] == 1) & (stock_daily['is_low'] == 1)
    all_stock_daily.loc[idx[:,stock],'probe_line_volume_expansion_low_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6).astype(int)

probe_line_volume_expansion_low_level_records = all_stock_daily[all_stock_daily['probe_line_volume_expansion_low_level'] == 1]
daily_probe_line_volume_expansion_low_level_stocks = probe_line_volume_expansion_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [59]:
# Probe Line with Volume Expansion Performance
result_exploratory_line_volume_expansion = backtest_strategy(daily_probe_line_volume_expansion_stocks, all_stock_price_daily, volume_price_state="Volume Expansion")
result_exploratory_line_volume_expansion

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion,Hold 05 days,0.26%,0.07,48.66%,-45.27%,0.20%,51.97%
1,Volume Expansion,Hold 10 days,0.55%,0.10,131.34%,-59.63%,0.29%,51.89%
2,Volume Expansion,Hold 20 days,1.14%,0.14,147.43%,-56.09%,0.30%,51.03%


In [60]:
# Probe Line with Low Level Performance
result_exploratory_line_low_level = backtest_strategy(daily_probe_line_low_level_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_exploratory_line_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.67%,0.08,71.91%,-47.33%,0.38%,53.04%
1,Low Level,Hold 10 days,1.34%,0.12,108.40%,-88.55%,0.34%,51.59%
2,Low Level,Hold 20 days,2.52%,0.18,292.08%,-41.88%,0.16%,50.52%


In [61]:
# Probe Line with Volume Expansion + Low Level Performance
result_exploratory_line_volume_expansion_low_level = backtest_strategy(daily_probe_line_volume_expansion_low_level_stocks, all_stock_price_daily, volume_price_state="Volume Expansion + Low Level")
result_exploratory_line_volume_expansion_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + Low Level,Hold 05 days,1.16%,0.10,98.94%,-47.33%,0.47%,53.58%
1,Volume Expansion + Low Level,Hold 10 days,2.01%,0.14,124.48%,-43.12%,0.38%,51.78%
2,Volume Expansion + Low Level,Hold 20 days,3.33%,0.22,249.65%,-42.90%,-0.11%,49.19%


### 3.2.2 Breakthrough Pattern

The Breakthrough Pattern consists of a long-bodied bullish candle followed by a candle with a long lower shadow. It is an effective bullish candlestick pattern discovered through the summarization of historical trading experience.

Feature Details:

1） Current day's return (Returns) > 0；

2） Current day's K-line lower shadow ratio (Lower Shadow) ≥ 50%；

3） Previous day's K-line upper shadow ratio (Upper Shadow) < 10%；

4） Previous day's K-line lower shadow ratio (Lower Shadow) < 10%；

5） Previous day's return (Returns) ≥ 0.5%；

6） No gap openings occurred on either day.

<div align="center">

<img src="./img/9.png" width="800">

</div>

In [62]:
# Continuation Breakout Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day return > 0
    condition_1 = stock_daily['pct_chg'] > 0
    
    # Condition 2: Current day lower shadow ratio ≥ 50%
    condition_2 = stock_daily['lower_shadow'] >= 0.5
    
    # Condition 3: Previous day upper shadow ratio < 10%
    condition_3 = stock_daily['upper_shadow'].shift(1) < 0.1
    
    # Condition 4: Previous day lower shadow ratio < 10%
    condition_4 = stock_daily['lower_shadow'].shift(1) < 0.1
    
    # Condition 5: Previous day return ≥ 0.5%
    condition_5 = stock_daily['pct_chg'].shift(1) >= 0.5
    
    # Condition 6: No gaps on either day
    condition_6 = ((stock_daily['open'] >= stock_daily['low'].shift(1)) & 
                   (stock_daily['open'] <= stock_daily['high'].shift(1)) &
                   (stock_daily['open'].shift(1) >= stock_daily['low'].shift(2)) & 
                   (stock_daily['open'].shift(1) <= stock_daily['high'].shift(2)))

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6

    all_stock_daily.loc[idx[:,stock],'continuation_breakout'] = condition.astype(int)

# Filter all continuation breakout records
continuation_breakout_records = all_stock_daily[all_stock_daily['continuation_breakout'] == 1]

# Group by date and collect stock codes
daily_continuation_breakout_stocks = continuation_breakout_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [63]:
# Continuation Breakout Pattern Performance
result_continuation_breakout = backtest_strategy(daily_continuation_breakout_stocks, all_stock_price_daily, volume_price_state="Continuation Breakout")
result_continuation_breakout

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Continuation Breakout,Hold 05 days,0.26%,0.08,106.56%,-44.49%,0.47%,53.81%
1,Continuation Breakout,Hold 10 days,0.80%,0.12,129.48%,-58.68%,0.96%,54.31%
2,Continuation Breakout,Hold 20 days,1.49%,0.17,184.27%,-54.55%,0.38%,51.27%


In [65]:
# Continuation Breakout + Low Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day return > 0
    condition_1 = stock_daily['pct_chg'] > 0
    
    # Condition 2: Current day lower shadow ratio ≥ 50%
    condition_2 = stock_daily['lower_shadow'] >= 0.5
    
    # Condition 3: Previous day upper shadow ratio < 10%
    condition_3 = stock_daily['upper_shadow'].shift(1) < 0.1
    
    # Condition 4: Previous day lower shadow ratio < 10%
    condition_4 = stock_daily['lower_shadow'].shift(1) < 0.1
    
    # Condition 5: Previous day return ≥ 0.5%
    condition_5 = stock_daily['pct_chg'].shift(1) >= 0.5
    
    # Condition 6: No gaps on either day
    condition_6 = ((stock_daily['open'] >= stock_daily['low'].shift(1)) & 
                   (stock_daily['open'] <= stock_daily['high'].shift(1)) &
                   (stock_daily['open'].shift(1) >= stock_daily['low'].shift(2)) & 
                   (stock_daily['open'].shift(1) <= stock_daily['high'].shift(2)))
    
    # Condition 7: Low price level
    condition_7 = (stock_daily['is_low'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7

    all_stock_daily.loc[idx[:,stock],'continuation_breakout_low_level'] = condition.astype(int)

# Filter all continuation breakout + low level records
continuation_breakout_low_level_records = all_stock_daily[all_stock_daily['continuation_breakout_low_level'] == 1]

# Group by date and collect stock codes
daily_continuation_breakout_low_level_stocks = continuation_breakout_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [67]:
# Continuation Breakout with Low Level Performance
result_continuation_breakout_low_Level = backtest_strategy(daily_continuation_breakout_low_level_stocks, all_stock_price_daily, volume_price_state="Continuation Breakout + Low Level")
result_continuation_breakout_low_Level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Continuation Breakout + Low Level,Hold 05 days,1.07%,0.11,64.27%,-45.45%,0.47%,53.08%
1,Continuation Breakout + Low Level,Hold 10 days,1.69%,0.17,126.97%,-45.45%,0.61%,51.84%
2,Continuation Breakout + Low Level,Hold 20 days,4.67%,0.26,241.00%,-51.33%,-0.20%,49.29%


### 3.2.3 Shooting Star Pattern

The Shooting Star is an iconic top reversal candlestick pattern that typically appears at the end of an uptrend, reflecting exhaustion of buying pressure and the beginning of selling pressure.

Feature Details:

1) Current day's K-line is bearish (Yin);

2) Current day's upper shadow ratio (Upper Shadow) ≥ 40%;

3) Current day's lower shadow ratio (Lower Shadow) < 5%;

4) Current day's body ratio (Body) ≤ 40%;

5) Current day's closing price (Close) < previous day's opening price (Open);

6) Previous day's K-line is bullish (Yang);

7) Previous day's upper shadow ratio (Upper Shadow) < 30%;

8) Previous day's lower shadow ratio (Lower Shadow) < 30%.

<div align="center">
<img src="./img/10.png" width="800">
</div>

In [68]:
# Shooting Star Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bearish (Yin)
    condition_1 = (stock_daily['is_bullish'] == 0)
    
    # Condition 2: Current day upper shadow ratio ≥ 40%
    condition_2 = (stock_daily['upper_shadow'] >= 0.4)
    
    # Condition 3: Current day lower shadow ratio < 5%
    condition_3 = (stock_daily['lower_shadow'] < 0.05)
    
    # Condition 4: Current day body ratio ≤ 40%
    condition_4 = (stock_daily['body'] <= 0.4)
    
    # Condition 5: Current day close < previous day open
    condition_5 = (stock_daily['close'] < stock_daily['open'].shift(1))
    
    # Condition 6: Previous day is bullish (Yang)
    condition_6 = (stock_daily['is_bullish'].shift(1) == 1)
    
    # Condition 7: Previous day upper shadow ratio < 30%
    condition_7 = (stock_daily['upper_shadow'].shift(1) < 0.3)
    
    # Condition 8: Previous day lower shadow ratio < 30%
    condition_8 = (stock_daily['lower_shadow'].shift(1) < 0.3)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7 & condition_8

    all_stock_daily.loc[idx[:,stock],'shooting_star'] = condition.astype(int)

# Filter all shooting star records
shooting_star_records = all_stock_daily[all_stock_daily['shooting_star'] == 1]

# Group by date and collect stock codes
daily_shooting_star_stocks = shooting_star_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [69]:
# Shooting Star Pattern Performance
result_shooting_star = backtest_strategy(daily_shooting_star_stocks, all_stock_price_daily, volume_price_state="Shooting Star")
result_shooting_star

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Shooting Star,Hold 05 days,-1.62%,0.15,106.79%,-46.81%,-2.47%,39.56%
1,Shooting Star,Hold 10 days,-1.29%,0.21,175.97%,-74.25%,-3.88%,40.22%
2,Shooting Star,Hold 20 days,1.85%,0.42,897.98%,-84.47%,-2.91%,43.10%


In [70]:
# Shooting Star Pattern with Volume Expansion
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] >= 0.4)
    condition_3 = (stock_daily['lower_shadow'] < 0.05)
    condition_4 = (stock_daily['body'] <= 0.4)
    condition_5 = (stock_daily['close'] < stock_daily['open'].shift(1))
    condition_6 = (stock_daily['is_bullish'].shift(1) == 1)
    condition_7 = (stock_daily['upper_shadow'].shift(1) < 0.3)
    condition_8 = (stock_daily['lower_shadow'].shift(1) < 0.3)
    condition_9 = (stock_daily['volume_state'] == 1)
    all_stock_daily.loc[idx[:,stock],'shooting_star_volume_expansion'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7 & condition_8 & condition_9).astype(int)

shooting_star_volume_expansion_records = all_stock_daily[all_stock_daily['shooting_star_volume_expansion'] == 1]
daily_shooting_star_volume_expansion_stocks = shooting_star_volume_expansion_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Shooting Star with Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] >= 0.4)
    condition_3 = (stock_daily['lower_shadow'] < 0.05)
    condition_4 = (stock_daily['body'] <= 0.4)
    condition_5 = (stock_daily['close'] < stock_daily['open'].shift(1))
    condition_6 = (stock_daily['is_bullish'].shift(1) == 1)
    condition_7 = (stock_daily['upper_shadow'].shift(1) < 0.3)
    condition_8 = (stock_daily['lower_shadow'].shift(1) < 0.3)
    condition_9 = (stock_daily['is_low'] == 1)
    all_stock_daily.loc[idx[:,stock],'shooting_star_low_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7 & condition_8 & condition_9).astype(int)

shooting_star_low_level_records = all_stock_daily[all_stock_daily['shooting_star_low_level'] == 1]
daily_shooting_star_low_level_stocks = shooting_star_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Shooting Star with Volume Expansion + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['upper_shadow'] >= 0.4)
    condition_3 = (stock_daily['lower_shadow'] < 0.05)
    condition_4 = (stock_daily['body'] <= 0.4)
    condition_5 = (stock_daily['close'] < stock_daily['open'].shift(1))
    condition_6 = (stock_daily['is_bullish'].shift(1) == 1)
    condition_7 = (stock_daily['upper_shadow'].shift(1) < 0.3)
    condition_8 = (stock_daily['lower_shadow'].shift(1) < 0.3)
    condition_9 = (stock_daily['volume_state'] == 1) & (stock_daily['is_low'] == 1)
    all_stock_daily.loc[idx[:,stock],'shooting_star_volume_expansion_low_level'] = (condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7 & condition_8 & condition_9).astype(int)

shooting_star_volume_expansion_low_level_records = all_stock_daily[all_stock_daily['shooting_star_volume_expansion_low_level'] == 1]
daily_shooting_star_volume_expansion_low_level_stocks = shooting_star_volume_expansion_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [71]:
# Shooting Star with Volume Expansion Performance:
result_shooting_star_volume_expansion = backtest_strategy(daily_shooting_star_volume_expansion_stocks, all_stock_price_daily, volume_price_state="Volume Expansion")
result_shooting_star_volume_expansion

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion,Hold 05 days,-0.73%,0.19,129.73%,-80.11%,-2.32%,41.60%
1,Volume Expansion,Hold 10 days,1.21%,0.41,803.03%,-84.47%,-2.35%,45.20%
2,Volume Expansion,Hold 20 days,3.74%,0.54,1096.97%,-83.24%,-2.29%,45.32%


In [72]:
# Shooting Star with Low Level Performance
result_shooting_star_low_level = backtest_strategy(daily_shooting_star_low_level_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_shooting_star_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.05%,0.18,108.53%,-60.04%,-0.98%,44.23%
1,Low Level,Hold 10 days,3.43%,0.35,433.33%,-64.39%,-1.75%,46.39%
2,Low Level,Hold 20 days,4.66%,0.38,418.30%,-65.76%,-2.56%,43.68%


In [73]:
# Shooting Star with Volume Expansion + Low Level Performance
result_shooting_star_volume_expansion_low_level = backtest_strategy(daily_shooting_star_volume_expansion_low_level_stocks, all_stock_price_daily, volume_price_state="Volume Expansion + Low Level")
result_shooting_star_volume_expansion_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + Low Level,Hold 05 days,0.65%,0.21,118.51%,-55.07%,-1.81%,43.24%
1,Volume Expansion + Low Level,Hold 10 days,6.44%,0.62,897.98%,-50.00%,-0.91%,46.61%
2,Volume Expansion + Low Level,Hold 20 days,12.53%,1.29,1895.96%,-58.68%,0.36%,50.62%


### 3.2.4 Bearish Engulfing Pattern

The Bearish Engulfing Pattern is typically considered a classic top reversal pattern, usually appearing at the end of an uptrend, reflecting exhaustion of buying pressure and the start of selling pressure.

Feature Details:

1） Current day's K-line is bearish (Yin);

2） Current day's opening price (Open) > previous day's closing price (Close);

3） Current day's closing price (Close) < previous day's opening price (Open);

4） Current day's body ratio (Body) ≥ 70%;

5） Previous day's K-line is bullish (Yang);

6） Previous day's body ratio (Body) ≥ 70%.

<div align="center">

<img src="./img/11.png" width="800">

</div>

In [74]:
# Bearish Engulfing Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bearish (Yin)
    condition_1 = (stock_daily['is_bullish'] == 0)
    
    # Condition 2: Current day open > previous day close
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    
    # Condition 3: Current day close < previous day open
    condition_3 = (stock_daily['close'] < stock_daily['open'].shift(1))
    
    # Condition 4: Current day body ratio ≥ 70%
    condition_4 = (stock_daily['body'] >= 0.7)
    
    # Condition 5: Previous day is bullish (Yang)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 1)
    
    # Condition 6: Previous day body ratio ≥ 70%
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)

    # Combine all conditions
    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6

    # Mark the pattern
    all_stock_daily.loc[idx[:,stock],'bearish_engulfing'] = condition.astype(int)

# Filter all bearish engulfing records
bearish_engulfing_records = all_stock_daily[all_stock_daily['bearish_engulfing'] == 1]

# Group by date and collect stock codes
daily_bearish_engulfing_stocks = bearish_engulfing_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [75]:
# Bearish Engulfing Pattern Performance
result_bearish_engulfing = backtest_strategy(daily_bearish_engulfing_stocks, all_stock_price_daily, volume_price_state="Bearish Engulfing")
result_bearish_engulfing

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Bearish Engulfing,Hold 05 days,-0.38%,0.09,87.96%,-48.39%,-0.48%,46.18%
1,Bearish Engulfing,Hold 10 days,-0.22%,0.12,151.55%,-58.50%,-0.61%,46.17%
2,Bearish Engulfing,Hold 20 days,0.48%,0.16,199.48%,-65.42%,-0.45%,48.16%


In [76]:
# Bearish Engulfing Pattern with Price-Volume States

# Combine with Low Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bearish
    condition_1 = (stock_daily['is_bullish'] == 0)
    # Condition 2: Current open > previous close
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    # Condition 3: Current close < previous open
    condition_3 = (stock_daily['close'] < stock_daily['open'].shift(1))
    # Condition 4: Current body ratio >= 70%
    condition_4 = (stock_daily['body'] >= 0.7)
    # Condition 5: Previous day is bullish
    condition_5 = (stock_daily['is_bullish'].shift(1) == 1)
    # Condition 6: Previous body ratio >= 70%
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    # Condition 7: Low price level
    condition_7 = (stock_daily['is_low'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bearish_engulfing_low_level'] = condition.astype(int)

bearish_engulfing_low_level_records = all_stock_daily[all_stock_daily['bearish_engulfing_low_level'] == 1]
daily_bearish_engulfing_low_level_stocks = bearish_engulfing_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with Top Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    condition_3 = (stock_daily['close'] < stock_daily['open'].shift(1))
    condition_4 = (stock_daily['body'] >= 0.7)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 1)
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    condition_7 = (stock_daily['is_top'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bearish_engulfing_top_level'] = condition.astype(int)

bearish_engulfing_top_level_records = all_stock_daily[all_stock_daily['bearish_engulfing_top_level'] == 1]
daily_bearish_engulfing_top_level_stocks = bearish_engulfing_top_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with Volume Contraction + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    condition_3 = (stock_daily['close'] < stock_daily['open'].shift(1))
    condition_4 = (stock_daily['body'] >= 0.7)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 1)
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    condition_7 = (stock_daily['volume_state'] == 0) & (stock_daily['is_low'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bearish_engulfing_vol_contraction_low'] = condition.astype(int)

bearish_engulfing_vol_contraction_low_records = all_stock_daily[all_stock_daily['bearish_engulfing_vol_contraction_low'] == 1]
daily_bearish_engulfing_vol_contraction_low_stocks = bearish_engulfing_vol_contraction_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Combine with Volume Contraction + Top Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_1 = (stock_daily['is_bullish'] == 0)
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    condition_3 = (stock_daily['close'] < stock_daily['open'].shift(1))
    condition_4 = (stock_daily['body'] >= 0.7)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 1)
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    condition_7 = (stock_daily['volume_state'] == 0) & (stock_daily['is_top'] == 1)

    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bearish_engulfing_vol_contraction_top'] = condition.astype(int)

bearish_engulfing_vol_contraction_top_records = all_stock_daily[all_stock_daily['bearish_engulfing_vol_contraction_top'] == 1]
daily_bearish_engulfing_vol_contraction_top_stocks = bearish_engulfing_vol_contraction_top_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [77]:
# Bearish Engulfing with Low Level Performance
result_bearish_engulfing_low_level = backtest_strategy(daily_bearish_engulfing_low_level_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_bearish_engulfing_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.68%,0.11,129.90%,-30.48%,-0.37%,46.75%
1,Low Level,Hold 10 days,1.27%,0.15,129.38%,-36.35%,-1.04%,45.44%
2,Low Level,Hold 20 days,3.56%,0.27,505.15%,-49.50%,-0.86%,47.97%


In [78]:
# Bearish Engulfing with Top Level Performance
result_bearish_engulfing_top_level = backtest_strategy(daily_bearish_engulfing_top_level_stocks, all_stock_price_daily, volume_price_state="Top Level")
result_bearish_engulfing_top_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Top Level,Hold 05 days,-1.37%,0.15,75.79%,-68.91%,-2.45%,40.54%
1,Top Level,Hold 10 days,-1.17%,0.18,116.25%,-61.58%,-2.58%,43.27%
2,Top Level,Hold 20 days,-0.29%,0.25,180.47%,-55.64%,-4.74%,41.05%


In [79]:
# Bearish Engulfing with Volume Contraction + Low Level Performance
result_bearish_engulfing_volume_contraction_low_level = backtest_strategy(daily_bearish_engulfing_vol_contraction_low_stocks, all_stock_price_daily, volume_price_state="Volume Contraction + Low Level")
result_bearish_engulfing_volume_contraction_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Contraction + Low Level,Hold 05 days,0.55%,0.12,60.17%,-28.30%,-1.28%,44.34%
1,Volume Contraction + Low Level,Hold 10 days,0.94%,0.17,87.06%,-37.17%,-1.36%,44.26%
2,Volume Contraction + Low Level,Hold 20 days,2.72%,0.26,147.24%,-48.75%,-2.10%,44.51%


In [80]:
# Bearish Engulfing with Volume Contraction + Top Level Performance
result_bearish_engulfing_volume_contraction_top_level = backtest_strategy(daily_bearish_engulfing_vol_contraction_top_stocks, all_stock_price_daily, volume_price_state="Volume Contraction + Top Level")
result_bearish_engulfing_volume_contraction_top_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Contraction + Top Level,Hold 05 days,-0.33%,0.17,60.17%,-58.79%,-0.93%,46.46%
1,Volume Contraction + Top Level,Hold 10 days,1.77%,0.29,202.29%,-60.27%,-2.42%,46.28%
2,Volume Contraction + Top Level,Hold 20 days,4.03%,0.39,245.93%,-66.59%,-4.74%,44.32%


### 3.2.5 Bullish Engulfing Pattern

The Bullish Engulfing Pattern is a classic bottom reversal pattern, typically appearing at the end of a downtrend, reflecting strong buying pressure overpowering selling pressure.

Feature Details:

1) Current day's K-line is bullish (Yang);

2) Current day's opening price (Open) < previous day's closing price (Close);

3) Current day's closing price (Close) > previous day's opening price (Open);

4) Current day's body ratio (Body) ≥ 70%;

5) Previous day's K-line is bearish (Yin);

6) Previous day's body ratio (Body) ≥ 70%.

<div align="center">

<img src="./img/12.png" width="800">

</div>

<div align="center">

<img src="./img/13.png" width="800">

</div>

In [81]:
# Bullish Engulfing Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bullish (Close > Open)
    condition_1 = (stock_daily['is_bullish'] == 1)
    
    # Condition 2: Current open < previous close
    condition_2 = (stock_daily['open'] < stock_daily['close'].shift(1))
    
    # Condition 3: Current close > previous open
    condition_3 = (stock_daily['close'] > stock_daily['open'].shift(1))
    
    # Condition 4: Current body ratio >= 70%
    condition_4 = (stock_daily['body'] >= 0.7)
    
    # Condition 5: Previous day is bearish (Close < Open)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 0)
    
    # Condition 6: Previous body ratio >= 70%
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    
    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6
    
    all_stock_daily.loc[idx[:,stock],'bullish_engulfing'] = condition.astype(int)

# Filter bullish engulfing records
bullish_engulfing_records = all_stock_daily[all_stock_daily['bullish_engulfing'] == 1]

# Group by date and collect stock codes
daily_bullish_engulfing_stocks = bullish_engulfing_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [82]:
# Bullish Engulfing Pattern Performance
result_bullish_engulfing = backtest_strategy(daily_bullish_engulfing_stocks, all_stock_price_daily, volume_price_state="Bullish Engulfing")
result_bullish_engulfing

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Bullish Engulfing,Hold 05 days,-0.26%,0.08,77.23%,-53.33%,-0.34%,46.82%
1,Bullish Engulfing,Hold 10 days,-0.26%,0.10,93.08%,-55.46%,-0.54%,47.30%
2,Bullish Engulfing,Hold 20 days,0.11%,0.14,99.02%,-62.91%,-1.06%,45.91%


In [83]:
# Bullish Engulfing with Volume Expansion
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_1 = (stock_daily['is_bullish'] == 1)
    condition_2 = (stock_daily['open'] < stock_daily['close'].shift(1))
    condition_3 = (stock_daily['close'] > stock_daily['open'].shift(1))
    condition_4 = (stock_daily['body'] >= 0.7)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 0)
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    condition_7 = (stock_daily['volume_state'] == 1)
    
    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bullish_engulfing_volume_expansion'] = condition.astype(int)

bullish_engulfing_volume_expansion_records = all_stock_daily[all_stock_daily['bullish_engulfing_volume_expansion'] == 1]
daily_bullish_engulfing_volume_expansion_stocks = bullish_engulfing_volume_expansion_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Bullish Engulfing with Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_1 = (stock_daily['is_bullish'] == 1)
    condition_2 = (stock_daily['open'] < stock_daily['close'].shift(1))
    condition_3 = (stock_daily['close'] > stock_daily['open'].shift(1))
    condition_4 = (stock_daily['body'] >= 0.7)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 0)
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    condition_7 = (stock_daily['is_low'] == 1)
    
    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bullish_engulfing_low_level'] = condition.astype(int)

bullish_engulfing_low_level_records = all_stock_daily[all_stock_daily['bullish_engulfing_low_level'] == 1]
daily_bullish_engulfing_low_level_stocks = bullish_engulfing_low_level_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Bullish Engulfing with Volume Expansion + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_1 = (stock_daily['is_bullish'] == 1)
    condition_2 = (stock_daily['open'] < stock_daily['close'].shift(1))
    condition_3 = (stock_daily['close'] > stock_daily['open'].shift(1))
    condition_4 = (stock_daily['body'] >= 0.7)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 0)
    condition_6 = (stock_daily['body'].shift(1) >= 0.7)
    condition_7 = (stock_daily['volume_state'] == 1) & (stock_daily['is_low'] == 1)
    
    condition = condition_1 & condition_2 & condition_3 & condition_4 & condition_5 & condition_6 & condition_7
    all_stock_daily.loc[idx[:,stock],'bullish_engulfing_vol_exp_low'] = condition.astype(int)

bullish_engulfing_vol_exp_low_records = all_stock_daily[all_stock_daily['bullish_engulfing_vol_exp_low'] == 1]
daily_bullish_engulfing_vol_exp_low_stocks = bullish_engulfing_vol_exp_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [85]:
# Bullish Engulfing with Volume Expansion Performance
result_bullish_engulfing_volume_expansion =  backtest_strategy(daily_bullish_engulfing_volume_expansion_stocks, all_stock_price_daily, volume_price_state="Volume Expansion")
result_bullish_engulfing_volume_expansion

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion,Hold 05 days,-0.35%,0.08,50.90%,-51.48%,-0.30%,46.87%
1,Volume Expansion,Hold 10 days,-0.36%,0.10,89.95%,-55.15%,-0.61%,46.03%
2,Volume Expansion,Hold 20 days,0.28%,0.15,118.18%,-60.73%,-1.06%,45.32%


In [86]:
# Bullish Engulfing with Low Level Performance
result_bullish_engulfing_low_level = backtest_strategy(daily_bullish_engulfing_low_level_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_bullish_engulfing_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.67%,0.11,117.33%,-38.64%,0.04%,50.15%
1,Low Level,Hold 10 days,1.46%,0.16,135.24%,-40.00%,-0.19%,49.31%
2,Low Level,Hold 20 days,1.89%,0.18,91.45%,-52.76%,-0.79%,47.42%


In [87]:
# Bullish Engulfing with Volume Expansion + Low Level Performance
result_bullish_engulfing_volume_expansion_low_level = backtest_strategy(daily_bullish_engulfing_vol_exp_low_stocks, all_stock_price_daily, volume_price_state="Volume Expansion + Low Level")
result_bullish_engulfing_volume_expansion_low_level

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + Low Level,Hold 05 days,1.07%,0.13,174.69%,-42.38%,0.00%,49.95%
1,Volume Expansion + Low Level,Hold 10 days,1.83%,0.17,132.00%,-44.72%,-0.67%,47.34%
2,Volume Expansion + Low Level,Hold 20 days,2.55%,0.21,137.74%,-53.22%,-1.06%,46.89%


### 3.2.6 Rising Sun Pattern

The Rising Sun Pattern is a classic bottom reversal pattern, typically appearing at the end of prolonged downtrends, reflecting exhaustion of selling pressure and strong buying momentum.

Feature Details:

1） Current day's K-line is bullish (Yang);

2） Current day's opening price (Open) > previous day's closing price (Close);

3） Current day's closing price (Close) > previous day's opening price (Open);

4） Current day's return (Returns) ≥ 0.5%;

5） Current day's upper shadow ratio (Upper Shadow) ≤ 25%;

6） Current day's lower shadow ratio (Lower Shadow) ≤ 25%;

7） Previous day's K-line is bearish (Yin);

8） Previous day's return (Returns) ≤ -0.5%;

9） Previous day's upper shadow ratio (Upper Shadow) ≤ 25%;

10） Previous day's lower shadow ratio (Lower Shadow) ≤ 25%.

<div align="center">

<img src="./img/14.png" width="800">

</div>

In [88]:
# Rising Sun Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bullish (Yang)
    condition_1 = (stock_daily['is_bullish'] == 1)
    
    # Condition 2: Current open > previous close
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    
    # Condition 3: Current close > previous open
    condition_3 = (stock_daily['close'] > stock_daily['open'].shift(1))
    
    # Condition 4: Current return ≥ 0.5%
    condition_4 = (stock_daily['pct_chg'] >= 0.5)
    
    # Condition 5: Current upper shadow ≤ 25%
    condition_5 = (stock_daily['upper_shadow'] <= 0.25)
    
    # Condition 6: Current lower shadow ≤ 25%
    condition_6 = (stock_daily['lower_shadow'] <= 0.25)
    
    # Condition 7: Previous day is bearish (Yin)
    condition_7 = (stock_daily['is_bullish'].shift(1) == 0)
    
    # Condition 8: Previous return ≤ -0.5%
    condition_8 = (stock_daily['pct_chg'].shift(1) <= -0.5)
    
    # Condition 9: Previous upper shadow ≤ 25%
    condition_9 = (stock_daily['upper_shadow'].shift(1) <= 0.25)
    
    # Condition 10: Previous lower shadow ≤ 25%
    condition_10 = (stock_daily['lower_shadow'].shift(1) <= 0.25)

    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & condition_6 & 
        condition_7 & condition_8 & condition_9 & condition_10
    )

    all_stock_daily.loc[idx[:,stock],'rising_sun'] = condition.astype(int)

# Filter rising sun records
rising_sun_records = all_stock_daily[all_stock_daily['rising_sun'] == 1]

# Group by date and collect stock codes
daily_rising_sun_stocks = rising_sun_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [89]:
# Rising Sun Pattern Performance
result_rising_sun = backtest_strategy(daily_rising_sun_stocks, all_stock_price_daily, volume_price_state="Rising Sun")
result_rising_sun

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Rising Sun,Hold 05 days,0.28%,0.07,49.96%,-56.04%,0.23%,52.12%
1,Rising Sun,Hold 10 days,0.30%,0.09,52.21%,-58.09%,0.12%,50.63%
2,Rising Sun,Hold 20 days,0.82%,0.13,89.16%,-62.61%,0.00%,50.00%


In [90]:
# Rising Sun Pattern with Volume Expansion
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Rising Sun pattern conditions
    condition_rising_sun = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['open'] > stock_daily['close'].shift(1)) &
        (stock_daily['close'] > stock_daily['open'].shift(1)) &
        (stock_daily['pct_chg'] >= 0.5) &
        (stock_daily['upper_shadow'] <= 0.25) &
        (stock_daily['lower_shadow'] <= 0.25) &
        (stock_daily['is_bullish'].shift(1) == 0) &
        (stock_daily['pct_chg'].shift(1) <= -0.5) &
        (stock_daily['upper_shadow'].shift(1) <= 0.25) &
        (stock_daily['lower_shadow'].shift(1) <= 0.25)
    )

    # Volume expansion condition
    condition_volume = (stock_daily['volume_state'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'rising_sun_volume_up'] = (condition_rising_sun & condition_volume).astype(int)

rising_sun_volume_up_records = all_stock_daily[all_stock_daily['rising_sun_volume_up'] == 1]
daily_rising_sun_volume_up_stocks = rising_sun_volume_up_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Rising Sun with Low Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_rising_sun = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['open'] > stock_daily['close'].shift(1)) &
        (stock_daily['close'] > stock_daily['open'].shift(1)) &
        (stock_daily['pct_chg'] >= 0.5) &
        (stock_daily['upper_shadow'] <= 0.25) &
        (stock_daily['lower_shadow'] <= 0.25) &
        (stock_daily['is_bullish'].shift(1) == 0) &
        (stock_daily['pct_chg'].shift(1) <= -0.5) &
        (stock_daily['upper_shadow'].shift(1) <= 0.25) &
        (stock_daily['lower_shadow'].shift(1) <= 0.25)
    )

    condition_low = (stock_daily['is_low'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'rising_sun_low'] = (condition_rising_sun & condition_low).astype(int)

rising_sun_low_records = all_stock_daily[all_stock_daily['rising_sun_low'] == 1]
daily_rising_sun_low_stocks = rising_sun_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Rising Sun with Volume Expansion + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_rising_sun = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['open'] > stock_daily['close'].shift(1)) &
        (stock_daily['close'] > stock_daily['open'].shift(1)) &
        (stock_daily['pct_chg'] >= 0.5) &
        (stock_daily['upper_shadow'] <= 0.25) &
        (stock_daily['lower_shadow'] <= 0.25) &
        (stock_daily['is_bullish'].shift(1) == 0) &
        (stock_daily['pct_chg'].shift(1) <= -0.5) &
        (stock_daily['upper_shadow'].shift(1) <= 0.25) &
        (stock_daily['lower_shadow'].shift(1) <= 0.25)
    )
    
    condition_combo = (stock_daily['volume_state'] == 1) & (stock_daily['is_low'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'rising_sun_vol_up_low'] = (condition_rising_sun & condition_combo).astype(int)

rising_sun_vol_up_low_records = all_stock_daily[all_stock_daily['rising_sun_vol_up_low'] == 1]
daily_rising_sun_vol_up_low_stocks = rising_sun_vol_up_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [91]:
# Rising Sun with Volume Expansion Performance
result_rising_sun_volume_up = backtest_strategy(daily_rising_sun_volume_up_stocks, all_stock_price_daily, volume_price_state="Volume Expansion")
result_rising_sun_volume_up

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion,Hold 05 days,0.33%,0.07,55.06%,-45.45%,0.14%,51.17%
1,Volume Expansion,Hold 10 days,0.35%,0.10,71.78%,-58.09%,-0.09%,49.43%
2,Volume Expansion,Hold 20 days,0.91%,0.14,115.23%,-62.61%,-0.11%,49.66%


In [92]:
# Rising Sun with Low Level Performance
result_rising_sun_low = backtest_strategy(daily_rising_sun_low_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_rising_sun_low

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.72%,0.09,82.09%,-36.06%,0.19%,51.05%
1,Low Level,Hold 10 days,0.88%,0.12,132.72%,-41.69%,-0.25%,48.86%
2,Low Level,Hold 20 days,1.57%,0.17,114.89%,-63.66%,-0.87%,47.00%


In [93]:
# Rising Sun with Volume Expansion + Low Level Performance
result_rising_sun_volume_up_low = backtest_strategy(daily_rising_sun_vol_up_low_stocks, all_stock_price_daily, volume_price_state="Volume Expansion + Low Level")
result_rising_sun_volume_up_low

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + Low Level,Hold 05 days,0.71%,0.10,129.14%,-36.06%,0.00%,50.00%
1,Volume Expansion + Low Level,Hold 10 days,0.96%,0.13,114.29%,-40.43%,0.10%,50.31%
2,Volume Expansion + Low Level,Hold 20 days,1.58%,0.18,119.05%,-59.44%,-1.14%,47.13%


### 3.2.7 First Light Pattern

The First Light Pattern is an iconic bottom reversal formation that typically appears at the end of prolonged downtrends, reflecting exhaustion of selling pressure and tentative buying counterattacks.

Feature Details:

1) Current day's K-line is bullish (Yang);

2) Current day's opening price (Open) < previous day's closing price (Close);

3) Current day's closing price (Close) > average of previous day's opening price (Open) and closing price (Close);

4) Current day's return (Returns) ≥ 0.5%;

5) Current day's upper shadow ratio (Upper Shadow) ≤ 25%;

6) Current day's lower shadow ratio (Lower Shadow) ≤ 25%;

7) Previous day's K-line is bearish (Yin);

8) Previous day's return (Returns) ≤ -0.5%;

9) Previous day's upper shadow ratio (Upper Shadow) ≤ 25%;

10) Previous day's lower shadow ratio (Lower Shadow) ≤ 25%.

<div align="center">

<img src="./img/15.png" width="800">

</div>

In [94]:
# Dawn Breaking Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bullish (Yang)
    condition_1 = (stock_daily['is_bullish'] == 1)
    
    # Condition 2: Current open < previous close
    condition_2 = (stock_daily['open'] < stock_daily['close'].shift(1))
    
    # Condition 3: Current close > average of previous open and close
    condition_3 = (stock_daily['close'] > (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2)
    
    # Condition 4: Current return ≥ 0.5%
    condition_4 = (stock_daily['pct_chg'] >= 0.5)
    
    # Condition 5: Current upper shadow ≤ 25%
    condition_5 = (stock_daily['upper_shadow'] <= 0.25)
    
    # Condition 6: Current lower shadow ≤ 25%
    condition_6 = (stock_daily['lower_shadow'] <= 0.25)
    
    # Condition 7: Previous day is bearish (Yin)
    condition_7 = (stock_daily['is_bullish'].shift(1) == 0)
    
    # Condition 8: Previous return ≤ -0.5%
    condition_8 = (stock_daily['pct_chg'].shift(1) <= -0.5)
    
    # Condition 9: Previous upper shadow ≤ 25%
    condition_9 = (stock_daily['upper_shadow'].shift(1) <= 0.25)
    
    # Condition 10: Previous lower shadow ≤ 25%
    condition_10 = (stock_daily['lower_shadow'].shift(1) <= 0.25)
    
    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & condition_6 & 
        condition_7 & condition_8 & condition_9 & condition_10
    )
    
    all_stock_daily.loc[idx[:,stock],'dawn_breaking'] = condition.astype(int)

# Filter dawn breaking records
dawn_breaking_records = all_stock_daily[all_stock_daily['dawn_breaking'] == 1]

# Group by date and collect stock codes
daily_dawn_breaking_stocks = dawn_breaking_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [95]:
# Dawn Breaking Pattern Performance
result_dawn_breaking = backtest_strategy(daily_dawn_breaking_stocks, all_stock_price_daily, volume_price_state="Dawn Breaking")
result_dawn_breaking

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Dawn Breaking,Hold 05 days,-0.08%,0.07,46.20%,-48.88%,-0.14%,49.07%
1,Dawn Breaking,Hold 10 days,0.01%,0.09,50.67%,-48.98%,0.17%,50.82%
2,Dawn Breaking,Hold 20 days,0.40%,0.12,72.78%,-53.01%,-0.44%,48.09%


In [96]:
# Dawn Breaking with Volume Expansion
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Dawn Breaking pattern conditions
    condition_dawn = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['open'] < stock_daily['close'].shift(1)) &
        (stock_daily['close'] > (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2) &
        (stock_daily['pct_chg'] >= 0.5) &
        (stock_daily['upper_shadow'] <= 0.25) &
        (stock_daily['lower_shadow'] <= 0.25) &
        (stock_daily['is_bullish'].shift(1) == 0) &
        (stock_daily['pct_chg'].shift(1) <= -0.5) &
        (stock_daily['upper_shadow'].shift(1) <= 0.25) &
        (stock_daily['lower_shadow'].shift(1) <= 0.25)
    )
    
    # Volume expansion condition
    condition_volume = (stock_daily['volume_state'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'dawn_volume_up'] = (condition_dawn & condition_volume).astype(int)

dawn_volume_up_records = all_stock_daily[all_stock_daily['dawn_volume_up'] == 1]
daily_dawn_volume_up_stocks = dawn_volume_up_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Dawn Breaking with Low Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_dawn = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['open'] < stock_daily['close'].shift(1)) &
        (stock_daily['close'] > (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2) &
        (stock_daily['pct_chg'] >= 0.5) &
        (stock_daily['upper_shadow'] <= 0.25) &
        (stock_daily['lower_shadow'] <= 0.25) &
        (stock_daily['is_bullish'].shift(1) == 0) &
        (stock_daily['pct_chg'].shift(1) <= -0.5) &
        (stock_daily['upper_shadow'].shift(1) <= 0.25) &
        (stock_daily['lower_shadow'].shift(1) <= 0.25)
    )
    
    condition_low = (stock_daily['is_low'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'dawn_low'] = (condition_dawn & condition_low).astype(int)

dawn_low_records = all_stock_daily[all_stock_daily['dawn_low'] == 1]
daily_dawn_low_stocks = dawn_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Dawn Breaking with Volume Expansion + Low Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_dawn = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['open'] < stock_daily['close'].shift(1)) &
        (stock_daily['close'] > (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2) &
        (stock_daily['pct_chg'] >= 0.5) &
        (stock_daily['upper_shadow'] <= 0.25) &
        (stock_daily['lower_shadow'] <= 0.25) &
        (stock_daily['is_bullish'].shift(1) == 0) &
        (stock_daily['pct_chg'].shift(1) <= -0.5) &
        (stock_daily['upper_shadow'].shift(1) <= 0.25) &
        (stock_daily['lower_shadow'].shift(1) <= 0.25)
    )
    
    condition_combo = (stock_daily['volume_state'] == 1) & (stock_daily['is_low'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'dawn_vol_up_low'] = (condition_dawn & condition_combo).astype(int)

dawn_vol_up_low_records = all_stock_daily[all_stock_daily['dawn_vol_up_low'] == 1]
daily_dawn_vol_up_low_stocks = dawn_vol_up_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [97]:
# Dawn Breaking with Volume Expansion Performance
result_dawn_volume_up = backtest_strategy(daily_dawn_volume_up_stocks, all_stock_price_daily, volume_price_state="Volume Expansion")
result_dawn_volume_up

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion,Hold 05 days,-0.16%,0.07,57.27%,-50.23%,-0.21%,48.33%
1,Volume Expansion,Hold 10 days,-0.17%,0.10,62.47%,-53.31%,-0.11%,49.19%
2,Volume Expansion,Hold 20 days,0.34%,0.13,86.17%,-56.46%,-0.97%,46.00%


In [98]:
# Dawn Breaking with Low Level Performance
result_dawn_low = backtest_strategy(daily_dawn_low_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_dawn_low

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.34%,0.08,58.19%,-46.47%,-0.16%,48.64%
1,Low Level,Hold 10 days,0.80%,0.11,104.89%,-36.62%,-0.11%,49.16%
2,Low Level,Hold 20 days,1.47%,0.15,97.68%,-42.53%,-0.57%,47.80%


In [99]:
# Dawn Breaking with Volume Expansion + Low Level Performance
result_dawn_volume_up_low = backtest_strategy(daily_dawn_vol_up_low_stocks, all_stock_price_daily, volume_price_state="Volume Expansion + Low Level")
result_dawn_volume_up_low

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Expansion + Low Level,Hold 05 days,0.68%,0.09,76.46%,-34.58%,0.07%,50.55%
1,Volume Expansion + Low Level,Hold 10 days,1.23%,0.15,150.37%,-49.52%,-0.28%,48.05%
2,Volume Expansion + Low Level,Hold 20 days,2.09%,0.21,351.43%,-46.23%,-0.49%,47.06%


### 3.2.8 Dark Cloud Cover Pattern

The Dark Cloud Cover pattern is generally regarded as a top reversal pattern, usually appearing at the end of an uptrend or at the top of a sideways consolidation range, reflecting exhaustion of buying pressure and a counterattack by sellers.

Feature Details:

1) Current day's K-line is bearish (Yin);

2) Current day's opening price (Open) > previous day's closing price (Close);

3) Current day's closing price (Close) < average of previous day's opening price (Open) and closing price (Close);

4) Current day's upper shadow ratio (Upper Shadow) ≤ 10%;

5) Current day's lower shadow ratio (Lower Shadow) ≤ 10%;

6) Previous day's K-line is bullish (Yang);

7) Previous day's upper shadow ratio (Upper Shadow) ≤ 10%;

8) Previous day's lower shadow ratio (Lower Shadow) ≤ 10%.

<div align="center">

<img src="./img/16.png" width="800">

</div>

In [100]:
# Dark Cloud Cover Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bearish (Yin)
    condition_1 = (stock_daily['is_bullish'] == 0)
    
    # Condition 2: Current open > previous close
    condition_2 = (stock_daily['open'] > stock_daily['close'].shift(1))
    
    # Condition 3: Current close < average of previous open and close
    condition_3 = (stock_daily['close'] < (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2)
    
    # Condition 4: Current upper shadow ≤ 10%
    condition_4 = (stock_daily['upper_shadow'] <= 0.1)
    
    # Condition 5: Current lower shadow ≤ 10%
    condition_5 = (stock_daily['lower_shadow'] <= 0.1)
    
    # Condition 6: Previous day is bullish (Yang)
    condition_6 = (stock_daily['is_bullish'].shift(1) == 1)
    
    # Condition 7: Previous upper shadow ≤ 10%
    condition_7 = (stock_daily['upper_shadow'].shift(1) <= 0.1)
    
    # Condition 8: Previous lower shadow ≤ 10%
    condition_8 = (stock_daily['lower_shadow'].shift(1) <= 0.1)
    
    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & 
        condition_6 & condition_7 & condition_8
    )
    
    all_stock_daily.loc[idx[:,stock],'dark_cloud'] = condition.astype(int)
    
# Filter dark cloud cover records
dark_cloud_records = all_stock_daily[all_stock_daily['dark_cloud'] == 1]

# Group by date and collect stock codes
daily_dark_cloud_stocks = dark_cloud_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [101]:
# Dark Cloud Cover Pattern Performance
result_dark_cloud = backtest_strategy(daily_dark_cloud_stocks, all_stock_price_daily, volume_price_state="Dark Cloud Cover")
result_dark_cloud

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Dark Cloud Cover,Hold 05 days,-0.27%,0.12,163.42%,-58.12%,-0.78%,44.75%
1,Dark Cloud Cover,Hold 10 days,0.51%,0.17,209.02%,-57.33%,-1.05%,45.06%
2,Dark Cloud Cover,Hold 20 days,2.22%,0.23,282.73%,-65.82%,-1.25%,46.37%


In [102]:
# Dark Cloud Cover with High Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Dark Cloud Cover pattern conditions
    condition_dark_cloud = (
        (stock_daily['is_bullish'] == 0) &
        (stock_daily['open'] > stock_daily['close'].shift(1)) &
        (stock_daily['close'] < (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2) &
        (stock_daily['upper_shadow'] <= 0.1) &
        (stock_daily['lower_shadow'] <= 0.1) &
        (stock_daily['is_bullish'].shift(1) == 1) &
        (stock_daily['upper_shadow'].shift(1) <= 0.1) &
        (stock_daily['lower_shadow'].shift(1) <= 0.1)
    )
    
    # High price level condition
    condition_high = (stock_daily['is_high'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'dark_cloud_high'] = (condition_dark_cloud & condition_high).astype(int)

dark_cloud_high_records = all_stock_daily[all_stock_daily['dark_cloud_high'] == 1]
daily_dark_cloud_high_stocks = dark_cloud_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Dark Cloud Cover with High Level + Volume Contraction
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_dark_cloud = (
        (stock_daily['is_bullish'] == 0) &
        (stock_daily['open'] > stock_daily['close'].shift(1)) &
        (stock_daily['close'] < (stock_daily['open'].shift(1) + stock_daily['close'].shift(1))/2) &
        (stock_daily['upper_shadow'] <= 0.1) &
        (stock_daily['lower_shadow'] <= 0.1) &
        (stock_daily['is_bullish'].shift(1) == 1) &
        (stock_daily['upper_shadow'].shift(1) <= 0.1) &
        (stock_daily['lower_shadow'].shift(1) <= 0.1)
    )
    
    # Combined high level and volume contraction condition
    condition_combo = (stock_daily['is_high'] == 1) & (stock_daily['volume_state'] == 0)
    
    all_stock_daily.loc[idx[:,stock],'dark_cloud_high_vol_down'] = (condition_dark_cloud & condition_combo).astype(int)

dark_cloud_high_vol_down_records = all_stock_daily[all_stock_daily['dark_cloud_high_vol_down'] == 1]
daily_dark_cloud_high_vol_down_stocks = dark_cloud_high_vol_down_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [103]:
# Dark Cloud Cover with High Level Performance
result_dark_cloud_high = backtest_strategy(daily_dark_cloud_high_stocks, all_stock_price_daily, volume_price_state="High Level")
result_dark_cloud_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,High Level,Hold 05 days,-0.52%,0.15,88.09%,-67.55%,-1.24%,44.11%
1,High Level,Hold 10 days,0.88%,0.22,162.70%,-68.02%,-1.83%,44.84%
2,High Level,Hold 20 days,1.89%,0.30,225.64%,-64.46%,-3.35%,43.24%


In [104]:
# Dark Cloud Cover with High Level + Volume Contraction Performance
result_dark_cloud_high_volume_down = backtest_strategy(daily_dark_cloud_high_vol_down_stocks, all_stock_price_daily, volume_price_state="High Level + Volume Contraction")
result_dark_cloud_high_volume_down

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,High Level + Volume Contraction,Hold 05 days,0.79%,0.21,106.33%,-59.23%,-2.24%,43.06%
1,High Level + Volume Contraction,Hold 10 days,3.60%,0.31,248.93%,-54.13%,-1.88%,46.57%
2,High Level + Volume Contraction,Hold 20 days,9.33%,0.46,269.87%,-66.82%,-2.65%,46.63%


### 3.2.9 Heavy Rain Pattern

The Heavy Rain Pattern is a relatively typical top reversal pattern, usually appearing at the end of a significant uptrend, reflecting a sharp decrease in buying pressure and a strong counterattack by sellers.

Feature Details:

1) Current day's K-line is bearish (Yin);

2) Current day's closing price (Close) < previous day's opening price (Open);

3) Current day's upper shadow ratio (Upper Shadow) ≤ 10%;

4) Current day's lower shadow ratio (Lower Shadow) ≤ 10%;

5) Previous day's K-line is bullish (Yang);

6) Previous day's upper shadow ratio (Upper Shadow) ≤ 10%;

7) Previous day's lower shadow ratio (Lower Shadow) ≤ 10%.

<div align="center">

<img src="./img/17.png" width="800">

</div>

In [105]:
# Heavy Rain Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bearish (Yin)
    condition_1 = (stock_daily['is_bullish'] == 0)
    
    # Condition 2: Current close < previous open
    condition_2 = (stock_daily['close'] < stock_daily['open'].shift(1))
    
    # Condition 3: Current upper shadow ≤ 10%
    condition_3 = (stock_daily['upper_shadow'] <= 0.1)
    
    # Condition 4: Current lower shadow ≤ 10%
    condition_4 = (stock_daily['lower_shadow'] <= 0.1)
    
    # Condition 5: Previous day is bullish (Yang)
    condition_5 = (stock_daily['is_bullish'].shift(1) == 1)
    
    # Condition 6: Previous upper shadow ≤ 10%
    condition_6 = (stock_daily['upper_shadow'].shift(1) <= 0.1)
    
    # Condition 7: Previous lower shadow ≤ 10%
    condition_7 = (stock_daily['lower_shadow'].shift(1) <= 0.1)

    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & condition_6 & condition_7
    )
    
    all_stock_daily.loc[idx[:,stock],'heavy_rain'] = condition.astype(int)

# Filter heavy rain records
heavy_rain_records = all_stock_daily[all_stock_daily['heavy_rain'] == 1]

# Group by date and collect stock codes
daily_heavy_rain_stocks = heavy_rain_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [106]:
# Heavy Rain Pattern Performance
result_heavy_rain = backtest_strategy(daily_heavy_rain_stocks, all_stock_price_daily, volume_price_state="Heavy Rain")
result_heavy_rain

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Heavy Rain,Hold 05 days,-0.16%,0.10,55.99%,-57.89%,-0.65%,45.10%
1,Heavy Rain,Hold 10 days,0.90%,0.16,160.00%,-53.21%,-0.87%,45.97%
2,Heavy Rain,Hold 20 days,2.15%,0.22,200.23%,-50.18%,-1.42%,45.31%


In [107]:
# Heavy Rain with High Price Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Heavy Rain pattern conditions
    condition_heavy_rain = (
        (stock_daily['is_bullish'] == 0) &
        (stock_daily['close'] < stock_daily['open'].shift(1)) &
        (stock_daily['upper_shadow'] <= 0.1) &
        (stock_daily['lower_shadow'] <= 0.1) &
        (stock_daily['is_bullish'].shift(1) == 1) &
        (stock_daily['upper_shadow'].shift(1) <= 0.1) &
        (stock_daily['lower_shadow'].shift(1) <= 0.1)
    )
    
    # High price level condition
    condition_high = (stock_daily['is_high'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'heavy_rain_high'] = (condition_heavy_rain & condition_high).astype(int)

# Filter heavy rain with high level records
heavy_rain_high_records = all_stock_daily[all_stock_daily['heavy_rain_high'] == 1]

# Group by date and collect stock codes
daily_heavy_rain_high_stocks = heavy_rain_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [108]:
# Heavy Rain with High Level Performance
result_heavy_rain_high = backtest_strategy(daily_heavy_rain_high_stocks, all_stock_price_daily, volume_price_state="Heavy Rain + High Level")
result_heavy_rain_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Heavy Rain + High Level,Hold 05 days,-0.50%,0.16,130.31%,-57.58%,-1.86%,42.96%
1,Heavy Rain + High Level,Hold 10 days,1.11%,0.21,112.04%,-50.20%,-2.69%,44.43%
2,Heavy Rain + High Level,Hold 20 days,3.20%,0.33,310.26%,-53.89%,-2.40%,45.93%


### 3.2.10 Double Hammer Pattern

The Double Hammer Pattern consists of two consecutive hammer candlesticks and is generally considered a bottom breakout signal. However, during actual testing, it has been observed that this pattern often appears as a bull trap.

Feature Details:

1) Both current and previous day's K-line have upper shadow ratio (Upper Shadow) < 5%;

2) Both current and previous day's K-line have lower shadow ratio (Lower Shadow) ≥ 50%;

3) Both current and previous day's K-line have body ratio (Body) ≥ 5%;

4) The smaller value between current day's opening price (Open) and closing price (Close) is greater than the smaller value between previous day's opening price (Open) and closing price (Close).

<div align="center">

<img src="./img/18.png" width="800">

</div>

In [109]:
# Twin Hammers Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Both days upper shadow < 5%
    condition_1 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['upper_shadow'].shift(1) < 0.05)
    
    # Condition 2: Both days lower shadow ≥ 50%
    condition_2 = (stock_daily['lower_shadow'] >= 0.5) & (stock_daily['lower_shadow'].shift(1) >= 0.5)
    
    # Condition 3: Both days body ratio ≥ 5%
    condition_3 = (stock_daily['body'] >= 0.05) & (stock_daily['body'].shift(1) >= 0.05)
    
    # Condition 4: Min(current open, current close) > Min(previous open, previous close)
    condition_4 = (np.minimum(stock_daily['open'], stock_daily['close']) > 
                   np.minimum(stock_daily['open'].shift(1), stock_daily['close'].shift(1)))
    
    all_stock_daily.loc[idx[:,stock],'twin_hammers'] = (
        condition_1 & condition_2 & condition_3 & condition_4
    ).astype(int)

# Filter twin hammers records
twin_hammers_records = all_stock_daily[all_stock_daily['twin_hammers'] == 1]

# Group by date and collect stock codes
daily_twin_hammers_stocks = twin_hammers_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [110]:
# Twin Hammers Pattern Performance
result_twin_hammers = backtest_strategy(daily_twin_hammers_stocks, all_stock_price_daily, volume_price_state="Twin Hammers")
result_twin_hammers

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Twin Hammers,Hold 05 days,-0.12%,0.09,77.14%,-40.45%,-0.01%,49.31%
1,Twin Hammers,Hold 10 days,0.27%,0.13,127.30%,-62.95%,-0.12%,49.12%
2,Twin Hammers,Hold 20 days,1.02%,0.17,112.07%,-70.03%,-0.42%,48.08%


In [111]:
# Twin Hammers with High Price Level Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Twin Hammers pattern conditions
    condition_1 = (stock_daily['upper_shadow'] < 0.05) & (stock_daily['upper_shadow'].shift(1) < 0.05)
    condition_2 = (stock_daily['lower_shadow'] >= 0.5) & (stock_daily['lower_shadow'].shift(1) >= 0.5)
    condition_3 = (stock_daily['body'] >= 0.05) & (stock_daily['body'].shift(1) >= 0.05)
    condition_4 = (np.minimum(stock_daily['open'], stock_daily['close']) > 
                  np.minimum(stock_daily['open'].shift(1), stock_daily['close'].shift(1)))
    
    # High price level condition
    condition_high = (stock_daily['is_high'] == 1)

    all_stock_daily.loc[idx[:,stock],'twin_hammers_high'] = (
        condition_1 & condition_2 & condition_3 & condition_4 & condition_high
    ).astype(int)

# Filter twin hammers with high level records
twin_hammers_high_records = all_stock_daily[all_stock_daily['twin_hammers_high'] == 1]

# Group by date and collect stock codes
daily_twin_hammers_high_stocks = twin_hammers_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [112]:
# Twin Hammers with High Level Performance
result_twin_hammers_high = backtest_strategy(daily_twin_hammers_high_stocks, all_stock_price_daily, volume_price_state="Twin Hammers + High Level")
result_twin_hammers_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Twin Hammers + High Level,Hold 05 days,-0.21%,0.16,128.54%,-62.05%,-0.48%,47.61%
1,Twin Hammers + High Level,Hold 10 days,0.40%,0.20,168.88%,-70.03%,-0.40%,48.76%
2,Twin Hammers + High Level,Hold 20 days,0.50%,0.23,159.76%,-60.05%,-0.82%,48.15%


## 3.3 Multi-Candlestick Patterns

### 3.3.1 Evening Star Pattern

The Evening Star Pattern is an iconic top reversal pattern, typically appearing at the end of long-term uptrends or rapid upward movements, reflecting exhaustion of buying pressure and a counterattack by sellers.

Feature Details:

1) Current day's K-line is bearish (Yin);

2) Current day's body ratio (Body) ≥ 50%;

3) Current day's return (Returns) < 0;

3) Previous day's body ratio (Body) ≤ 5%;

4) Previous day's opening price (Open) > two-days-prior closing price (Close);

5) Two-days-prior K-line is bullish (Yang);

6) Two-days-prior body ratio (Body) ≥ 75%.

<div align="center">

<img src="./img/19.png" width="800">

</div>

In [113]:
# Evening Star Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Condition 1: Current day is bearish (Yin)
    condition_1 = (stock_daily['is_bullish'] == 0)
    
    # Condition 2: Current day body ratio ≥ 50%
    condition_2 = (stock_daily['body'] >= 0.5)
    
    # Condition 3: Current day return < 0
    condition_3 = (stock_daily['pct_chg'] < 0)
    
    # Condition 4: Previous day body ratio ≤ 5%
    condition_4 = (stock_daily['body'].shift(1) <= 0.05)
    
    # Condition 5: Previous day open > two-days-prior close
    condition_5 = (stock_daily['open'].shift(1) > stock_daily['close'].shift(2))
    
    # Condition 6: Two-days-prior is bullish (Yang)
    condition_6 = (stock_daily['is_bullish'].shift(2) == 1)
    
    # Condition 7: Two-days-prior body ratio ≥ 75%
    condition_7 = (stock_daily['body'].shift(2) >= 0.75)
    
    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & 
        condition_6 & condition_7
    )
    
    all_stock_daily.loc[idx[:,stock],'evening_star'] = condition.astype(int)

# Filter evening star records
evening_star_records = all_stock_daily[all_stock_daily['evening_star'] == 1]

# Group by date and collect stock codes
daily_evening_star_stocks = evening_star_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [114]:
# Evening Star Pattern Performance
result_evening_star = backtest_strategy(daily_evening_star_stocks, all_stock_price_daily, volume_price_state="Evening Star")
result_evening_star

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Evening Star,Hold 05 days,-1.39%,0.10,64.14%,-50.61%,-1.36%,42.01%
1,Evening Star,Hold 10 days,-1.03%,0.14,81.21%,-53.53%,-1.72%,43.38%
2,Evening Star,Hold 20 days,-0.05%,0.19,170.57%,-57.26%,-2.02%,43.27%


In [115]:
# Evening Star with Volume Contraction
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Evening Star pattern conditions
    condition_evening_star = (
        (stock_daily['is_bullish'] == 0) &
        (stock_daily['body'] >= 0.5) &
        (stock_daily['pct_chg'] < 0) &
        (stock_daily['body'].shift(1) <= 0.05) &
        (stock_daily['open'].shift(1) > stock_daily['close'].shift(2)) &
        (stock_daily['is_bullish'].shift(2) == 1) &
        (stock_daily['body'].shift(2) >= 0.75)
    )
    
    # Volume contraction condition
    condition_volume = (stock_daily['volume_state'] == 0)
    
    all_stock_daily.loc[idx[:,stock],'evening_star_vol_down'] = (condition_evening_star & condition_volume).astype(int)

evening_star_vol_down_records = all_stock_daily[all_stock_daily['evening_star_vol_down'] == 1]
daily_evening_star_vol_down_stocks = evening_star_vol_down_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Evening Star with High Price Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_evening_star = (
        (stock_daily['is_bullish'] == 0) &
        (stock_daily['body'] >= 0.5) &
        (stock_daily['pct_chg'] < 0) &
        (stock_daily['body'].shift(1) <= 0.05) &
        (stock_daily['open'].shift(1) > stock_daily['close'].shift(2)) &
        (stock_daily['is_bullish'].shift(2) == 1) &
        (stock_daily['body'].shift(2) >= 0.75)
    )
    
    # High price level condition
    condition_high = (stock_daily['is_high'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'evening_star_high'] = (condition_evening_star & condition_high).astype(int)

evening_star_high_records = all_stock_daily[all_stock_daily['evening_star_high'] == 1]
daily_evening_star_high_stocks = evening_star_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

# Evening Star with Volume Contraction + High Level
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_evening_star = (
        (stock_daily['is_bullish'] == 0) &
        (stock_daily['body'] >= 0.5) &
        (stock_daily['pct_chg'] < 0) &
        (stock_daily['body'].shift(1) <= 0.05) &
        (stock_daily['open'].shift(1) > stock_daily['close'].shift(2)) &
        (stock_daily['is_bullish'].shift(2) == 1) &
        (stock_daily['body'].shift(2) >= 0.75)
    )
    
    # Combined volume contraction and high level condition
    condition_combo = (stock_daily['volume_state'] == 0) & (stock_daily['is_high'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'evening_star_vol_down_high'] = (condition_evening_star & condition_combo).astype(int)

evening_star_vol_down_high_records = all_stock_daily[all_stock_daily['evening_star_vol_down_high'] == 1]
daily_evening_star_vol_down_high_stocks = evening_star_vol_down_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [116]:
# Evening Star with Volume Contraction Performance
result_evening_star_volume_down = backtest_strategy(daily_evening_star_vol_down_stocks, all_stock_price_daily, volume_price_state="Volume Contraction")
result_evening_star_volume_down

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Contraction,Hold 05 days,-0.48%,0.11,72.73%,-56.13%,-0.59%,46.93%
1,Volume Contraction,Hold 10 days,-0.16%,0.16,114.19%,-57.90%,-1.01%,46.29%
2,Volume Contraction,Hold 20 days,1.21%,0.22,273.80%,-65.08%,-1.54%,46.60%


In [117]:
# Evening Star with High Level Performance
result_evening_star_high = backtest_strategy(daily_evening_star_high_stocks, all_stock_price_daily, volume_price_state="High Level")
result_evening_star_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,High Level,Hold 05 days,-1.96%,0.13,80.97%,-60.68%,-2.15%,40.98%
1,High Level,Hold 10 days,-1.06%,0.19,124.34%,-60.46%,-2.90%,41.15%
2,High Level,Hold 20 days,0.12%,0.23,121.39%,-67.31%,-2.82%,43.43%


In [118]:
# Evening Star with Volume Contraction + High Level Performance
result_evening_star_volume_down_high = backtest_strategy(daily_evening_star_vol_down_high_stocks, all_stock_price_daily, volume_price_state="Volume Contraction + High Level")
result_evening_star_volume_down_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Volume Contraction + High Level,Hold 05 days,-0.70%,0.14,62.93%,-60.66%,-1.26%,45.99%
1,Volume Contraction + High Level,Hold 10 days,0.40%,0.21,163.10%,-62.10%,-2.22%,45.55%
2,Volume Contraction + High Level,Hold 20 days,1.77%,0.27,186.32%,-60.18%,-2.09%,44.63%


### 3.3.2 Abandoned Baby Pattern

The Abandoned Baby Pattern is a special case of the Evening Star Pattern, typically appearing at the end of long-term uptrends or rapid upward movements, reflecting exhaustion of buying pressure and a counterattack by sellers. Its effectiveness is generally greater than that of the standard Evening Star Pattern.

Feature Details:

1) Current day's K-line is bearish (Yin);

2) Current day's highest price (High) < previous day's lowest price (Low), forming a gap down;

3) Current day's return (Returns) < 0;

4) Previous day's body ratio (Body) ≤ 5%;

5) Previous day's opening price (Open) > two-days-prior highest price (High), forming a gap up open;

6) Two-days-prior K-line is bullish (Yang).

<div align="center">

<img src="./img/20.png" width="800">

</div>

In [119]:
# Abandoned Baby Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bearish (Yin)
    condition_1 = (stock_daily['is_bullish'] == 0)
    
    # Condition 2: Current high < previous low (gap down)
    condition_2 = (stock_daily['high'] < stock_daily['low'].shift(1))
    
    # Condition 3: Current return < 0
    condition_3 = (stock_daily['pct_chg'] < 0)
    
    # Condition 4: Previous day body ratio ≤ 5% (doji)
    condition_4 = (stock_daily['body'].shift(1) <= 0.05)
    
    # Condition 5: Previous day open > two-days-prior high (gap up)
    condition_5 = (stock_daily['open'].shift(1) > stock_daily['high'].shift(2))
    
    # Condition 6: Two-days-prior is bullish (Yang)
    condition_6 = (stock_daily['is_bullish'].shift(2) == 1)

    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & 
        condition_6
    )
    
    all_stock_daily.loc[idx[:,stock],'abandoned_baby'] = condition.astype(int)

# Filter abandoned baby records
abandoned_baby_records = all_stock_daily[all_stock_daily['abandoned_baby'] == 1]

# Group by date and collect stock codes
daily_abandoned_baby_stocks = abandoned_baby_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [120]:
# Abandoned Baby Pattern Performance
result_abandoned_baby = backtest_strategy(daily_abandoned_baby_stocks, all_stock_price_daily, volume_price_state="Abandoned Baby")
result_abandoned_baby

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Abandoned Baby,Hold 05 days,-1.49%,0.21,109.99%,-47.79%,-4.98%,36.21%
1,Abandoned Baby,Hold 10 days,-0.87%,0.28,172.91%,-57.78%,-4.95%,38.69%
2,Abandoned Baby,Hold 20 days,2.11%,0.39,248.60%,-50.73%,-4.48%,43.44%


In [121]:
# Abandoned Baby with High Price Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Abandoned Baby pattern conditions
    condition_abandoned_baby = (
        (stock_daily['is_bullish'] == 0) &                      # Bearish candle
        (stock_daily['high'] < stock_daily['low'].shift(1)) &    # Gap down
        (stock_daily['pct_chg'] < 0) &                          # Negative return
        (stock_daily['body'].shift(1) <= 0.05) &                # Previous day doji (body ≤5%)
        (stock_daily['open'].shift(1) > stock_daily['high'].shift(2)) &  # Previous day gap up
        (stock_daily['is_bullish'].shift(2) == 1)               # Two-days-prior bullish
    )
    
    # High price level condition
    condition_high = (stock_daily['is_high'] == 1)

    all_stock_daily.loc[idx[:,stock],'abandoned_baby_high'] = (
        condition_abandoned_baby & condition_high
    ).astype(int)

# Filter abandoned baby with high level records
abandoned_baby_high_records = all_stock_daily[all_stock_daily['abandoned_baby_high'] == 1]

# Group by date and collect stock codes
daily_abandoned_baby_high_stocks = abandoned_baby_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [122]:
# Abandoned Baby with High Level Performance
result_abandoned_baby_high = backtest_strategy(daily_abandoned_baby_high_stocks, all_stock_price_daily, volume_price_state="Abandoned Baby + High Level")
result_abandoned_baby_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Abandoned Baby + High Level,Hold 05 days,-4.08%,0.28,121.88%,-49.75%,-9.69%,30.92%
1,Abandoned Baby + High Level,Hold 10 days,1.40%,0.42,165.61%,-70.30%,-10.83%,35.62%
2,Abandoned Baby + High Level,Hold 20 days,-0.06%,0.48,295.86%,-66.46%,-9.02%,38.30%


### 3.3.3 Morning Star Pattern

The Morning Star Pattern is an iconic bottom reversal pattern, typically appearing at the end of long-term downtrends or rapid decline phases, reflecting exhaustion of selling pressure and a counterattack by buyers.

Feature Details:

1) Current day's K-line is bullish (Yang);

2) Current day's body ratio (Body) ≥ 50%;

3) Current day's return (Returns) > 0;

4) Current day's opening price (Open) < previous day's closing price;

5) Previous day's body ratio (Body) ≤ 5%;

6) Previous day's opening price (Open) < two-days-prior closing price (Close);

7) Two-days-prior K-line is bearish (Yin);

8) Two-days-prior body ratio (Body) ≥ 50%.

<div align="center">

<img src="./img/21.png" width="800">

</div>

In [123]:
# Morning Star Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Condition 1: Current day is bullish (Yang)
    condition_1 = (stock_daily['is_bullish'] == 1)
    
    # Condition 2: Current day body ratio ≥ 50%
    condition_2 = (stock_daily['body'] >= 0.5)
    
    # Condition 3: Current day return > 0
    condition_3 = (stock_daily['pct_chg'] > 0)
    
    # Condition 4: Current open < previous close
    condition_4 = (stock_daily['open'] < stock_daily['close'].shift(1))
    
    # Condition 5: Previous day body ratio ≤ 5% (doji)
    condition_5 = (stock_daily['body'].shift(1) <= 0.05)
    
    # Condition 6: Previous day open < two-days-prior close
    condition_6 = (stock_daily['open'].shift(1) < stock_daily['close'].shift(2))
    
    # Condition 7: Two-days-prior is bearish (Yin)
    condition_7 = (stock_daily['is_bullish'].shift(2) == 0)
    
    # Condition 8: Two-days-prior body ratio ≥ 50%
    condition_8 = (stock_daily['body'].shift(2) >= 0.5)

    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5 & 
        condition_6 & condition_7 & condition_8
    )

    all_stock_daily.loc[idx[:,stock],'morning_star'] = condition.astype(int)

# Filter morning star records
morning_star_records = all_stock_daily[all_stock_daily['morning_star'] == 1]

# Group by date and collect stock codes
daily_morning_star_stocks = morning_star_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [124]:
# Morning Star Pattern Performance
result_morning_star = backtest_strategy(daily_morning_star_stocks, all_stock_price_daily, volume_price_state="Morning Star")
result_morning_star

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Morning Star,Hold 05 days,0.08%,0.11,160.00%,-45.72%,-0.38%,47.68%
1,Morning Star,Hold 10 days,0.70%,0.15,194.73%,-56.23%,-0.52%,47.59%
2,Morning Star,Hold 20 days,1.72%,0.20,255.57%,-64.04%,-0.58%,47.74%


In [125]:
# Morning Star with Low Price Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Morning Star pattern conditions
    condition_morning_star = (
        (stock_daily['is_bullish'] == 1) &                      # Bullish candle
        (stock_daily['body'] >= 0.5) &                          # Body ≥ 50%
        (stock_daily['pct_chg'] > 0) &                          # Positive return
        (stock_daily['open'] < stock_daily['close'].shift(1)) &  # Open < previous close
        (stock_daily['body'].shift(1) <= 0.05) &                # Previous day doji (body ≤5%)
        (stock_daily['open'].shift(1) < stock_daily['close'].shift(2)) &  # Previous day open < two-days-prior close
        (stock_daily['is_bullish'].shift(2) == 0) &             # Two-days-prior bearish
        (stock_daily['body'].shift(2) >= 0.5)                   # Two-days-prior body ≥50%
    )

    # Low price level condition
    condition_low = (stock_daily['is_low'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'morning_star_low'] = (condition_morning_star & condition_low).astype(int)

# Morning Star with Low Level + Volume Expansion Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    condition_morning_star = (
        (stock_daily['is_bullish'] == 1) &
        (stock_daily['body'] >= 0.5) &
        (stock_daily['pct_chg'] > 0) &
        (stock_daily['open'] < stock_daily['close'].shift(1)) &
        (stock_daily['body'].shift(1) <= 0.05) &
        (stock_daily['open'].shift(1) < stock_daily['close'].shift(2)) &
        (stock_daily['is_bullish'].shift(2) == 0) &
        (stock_daily['body'].shift(2) >= 0.5)
    )
    
    # Combined low level and volume expansion condition
    condition_combo = (stock_daily['is_low'] == 1) & (stock_daily['volume_state'] == 1)
    
    all_stock_daily.loc[idx[:,stock],'morning_star_low_vol_up'] = (condition_morning_star & condition_combo).astype(int)

# Filter records
morning_star_low_records = all_stock_daily[all_stock_daily['morning_star_low'] == 1]
morning_star_low_vol_up_records = all_stock_daily[all_stock_daily['morning_star_low_vol_up'] == 1]

# Group by date and collect stock codes
daily_morning_star_low_stocks = morning_star_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)
daily_morning_star_low_vol_up_stocks = morning_star_low_vol_up_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [126]:
# Morning Star with Low Level Performance
result_morning_star_low = backtest_strategy(daily_morning_star_low_stocks, all_stock_price_daily, volume_price_state="Low Level")
result_morning_star_low

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level,Hold 05 days,0.97%,0.13,133.55%,-34.41%,-0.11%,49.33%
1,Low Level,Hold 10 days,1.87%,0.18,144.97%,-33.55%,-0.98%,45.91%
2,Low Level,Hold 20 days,2.68%,0.24,185.76%,-52.35%,-1.13%,45.76%


In [127]:
# Morning Star with Low Level + Volume Expansion Performance
result_morning_star_low_volume_up = backtest_strategy(daily_morning_star_low_vol_up_stocks, all_stock_price_daily, volume_price_state="Low Level + Volume Expansion")
result_morning_star_low_volume_up

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Low Level + Volume Expansion,Hold 05 days,1.01%,0.16,105.66%,-31.98%,-1.01%,44.44%
1,Low Level + Volume Expansion,Hold 10 days,1.60%,0.25,224.93%,-45.88%,-2.80%,42.50%
2,Low Level + Volume Expansion,Hold 20 days,2.71%,0.31,243.49%,-50.93%,-3.86%,42.13%


### 3.3.4 Three White Soldiers Pattern

The Three White Soldiers Pattern is generally regarded as a bottom reversal pattern, typically appearing at the end of long-term downtrends or after consolidation periods, reflecting gradually strengthening buying pressure and driving trend reversal.

Feature Details:

1) Current day's K-line is bullish (Yang);

2) Current day's body ratio (Body) ≥ 50%;

3) Current day's return (Returns) between 0% and 3%;

4) Previous day's K-line is bullish (Yang);

5) Previous day's body ratio (Body) ≥ 50%;

6) Previous day's return (Returns) between 0% and 3%;

7) Two-days-prior K-line is bullish (Yang);

8) Two-days-prior body ratio (Body) ≥ 50%;

9) Two-days-prior return (Returns) between 0% and 3%.

<div align="center">

<img src="./img/22.png" width="800">

</div>

In [128]:
# Three White Soldiers Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]
    
    # Condition 1: Current day is bullish (Yang)
    condition_1 = (stock_daily['is_bullish'] == 1)
    
    # Condition 2: Current day body ratio ≥ 50%
    condition_2 = (stock_daily['body'] >= 0.5)
    
    # Condition 3: Current day return between 0%-3%
    condition_3 = (stock_daily['pct_chg'].between(0, 3))
    
    # Condition 4: Previous day is bullish (Yang)
    condition_4 = (stock_daily['is_bullish'].shift(1) == 1)
    
    # Condition 5: Previous day body ratio ≥ 50%
    condition_5 = (stock_daily['body'].shift(1) >= 0.5)
    
    # Condition 6: Previous day return between 0%-3%
    condition_6 = (stock_daily['pct_chg'].shift(1).between(0, 3))
    
    # Condition 7: Two-days-prior is bullish (Yang)
    condition_7 = (stock_daily['is_bullish'].shift(2) == 1)
    
    # Condition 8: Two-days-prior body ratio ≥ 50%
    condition_8 = (stock_daily['body'].shift(2) >= 0.5)
    
    # Condition 9: Two-days-prior return between 0%-3%
    condition_9 = (stock_daily['pct_chg'].shift(2).between(0, 3))

    condition = (
        condition_1 & condition_2 & condition_3 &
        condition_4 & condition_5 & condition_6 &
        condition_7 & condition_8 & condition_9
    )

    all_stock_daily.loc[idx[:,stock],'three_white_soldiers'] = condition.astype(int)

# Filter three white soldiers records
three_white_soldiers_records = all_stock_daily[all_stock_daily['three_white_soldiers'] == 1]

# Group by date and collect stock codes
daily_three_white_soldiers_stocks = three_white_soldiers_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [129]:
# Three White Soldiers Pattern Performance
result_three_white_soldiers = backtest_strategy(daily_three_white_soldiers_stocks, all_stock_price_daily,volume_price_state="Three White Soldiers")
result_three_white_soldiers

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Three White Soldiers,Hold 05 days,0.55%,0.05,49.19%,-40.94%,0.43%,56.09%
1,Three White Soldiers,Hold 10 days,1.16%,0.07,52.98%,-52.68%,0.91%,58.90%
2,Three White Soldiers,Hold 20 days,2.28%,0.11,106.33%,-55.59%,1.38%,56.72%


In [130]:
# Three White Soldiers with Low Price Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Three White Soldiers pattern conditions
    condition_three_soldiers = (
        (stock_daily['is_bullish'] == 1) &                      # Current day bullish
        (stock_daily['body'] >= 0.5) &                          # Body ≥ 50%
        (stock_daily['pct_chg'].between(0, 3)) &                # Return 0%-3%
        (stock_daily['is_bullish'].shift(1) == 1) &             # Previous day bullish
        (stock_daily['body'].shift(1) >= 0.5) &                 # Previous body ≥ 50%
        (stock_daily['pct_chg'].shift(1).between(0, 3)) &       # Previous return 0%-3%
        (stock_daily['is_bullish'].shift(2) == 1) &             # Two-days-prior bullish
        (stock_daily['body'].shift(2) >= 0.5) &                 # Two-days-prior body ≥ 50%
        (stock_daily['pct_chg'].shift(2).between(0, 3))         # Two-days-prior return 0%-3%
    )

    # Low price level condition
    condition_low = (stock_daily['is_low'] == 1)

    all_stock_daily.loc[idx[:,stock],'three_soldiers_low'] = (
        condition_three_soldiers & condition_low
    ).astype(int)

# Filter three white soldiers with low level records
three_soldiers_low_records = all_stock_daily[all_stock_daily['three_soldiers_low'] == 1]

# Group by date and collect stock codes
daily_three_soldiers_low_stocks = three_soldiers_low_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [131]:
# Three White Soldiers with Low Level Performance
result_three_soldiers_low = backtest_strategy(daily_three_soldiers_low_stocks, all_stock_price_daily, volume_price_state="Three White Soldiers + Low Level")
result_three_soldiers_low

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Three White Soldiers + Low Level,Hold 05 days,1.02%,0.08,108.38%,-26.53%,0.71%,57.17%
1,Three White Soldiers + Low Level,Hold 10 days,1.22%,0.10,96.27%,-48.16%,0.83%,55.15%
2,Three White Soldiers + Low Level,Hold 20 days,2.98%,0.16,131.98%,-45.99%,0.96%,53.78%


### 3.3.5 Three Black Crows Pattern

The Three Black Crows Pattern is an iconic top reversal pattern, typically appearing at the end of long-term uptrends or below key resistance levels, reflecting continuously strengthening selling pressure that dominates the market.

Feature Details:

1) Current day, previous day, and two-days-prior K-lines are bearish (Yin);

2) Current day, previous day, and two-days-prior returns (Returns) ≤ -0.01;

3) Current day, previous day, and two-days-prior body ratios (Body) ≥ 75%;

4) Two-days-prior opening price (Open) > three-days-prior closing price (Close), forming a gap up;

5) Three-days-prior return (Returns) > 0.

<div align="center">

<img src="./img/23.png" width="800">

</div>

In [132]:
# Three Black Crows Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Condition 1: Current, previous and two-days-prior are bearish (Yin)
    condition_1 = (
        (stock_daily['is_bullish'] == 0) & 
        (stock_daily['is_bullish'].shift(1) == 0) & 
        (stock_daily['is_bullish'].shift(2) == 0)
    ) 
    
    # Condition 2: Current, previous and two-days-prior returns ≤ -0.01
    condition_2 = (
        (stock_daily['pct_chg'] <= -0.01) & 
        (stock_daily['pct_chg'].shift(1) <= -0.01) & 
        (stock_daily['pct_chg'].shift(2) <= -0.01)
    )
    
    # Condition 3: Current, previous and two-days-prior body ratios ≥ 75%
    condition_3 = (
        (stock_daily['body'] >= 0.75) & 
        (stock_daily['body'].shift(1) >= 0.75) & 
        (stock_daily['body'].shift(2) >= 0.75)
    )

    # Condition 4: Two-days-prior open > three-days-prior close (gap up)
    condition_4 = (stock_daily['open'].shift(2) > stock_daily['close'].shift(3))
    
    # Condition 5: Three-days-prior return > 0
    condition_5 = (stock_daily['pct_chg'].shift(3) > 0)
    
    condition = (
        condition_1 & condition_2 & condition_3 & 
        condition_4 & condition_5
    )

    all_stock_daily.loc[idx[:,stock],'three_black_crows'] = condition.astype(int)

# Filter three black crows records
three_black_crows_records = all_stock_daily[all_stock_daily['three_black_crows'] == 1]

# Group by date and collect stock codes
daily_three_black_crows_stocks = three_black_crows_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [133]:
# Three Black Crows Pattern Performance
result_three_black_crows = backtest_strategy(daily_three_black_crows_stocks, all_stock_price_daily, volume_price_state="Three Black Crows")
result_three_black_crows

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Three Black Crows,Hold 05 days,1.97%,0.16,126.08%,-35.91%,-0.76%,47.57%
1,Three Black Crows,Hold 10 days,2.40%,0.20,122.28%,-52.53%,-0.08%,49.48%
2,Three Black Crows,Hold 20 days,7.24%,0.34,210.06%,-51.33%,0.39%,50.42%


In [134]:
# Three Black Crows with High Price Level Pattern Identification
for stock in all_stocks:
    stock_daily = all_stock_daily.loc[idx[:,stock],:]

    # Three Black Crows pattern conditions
    condition_crows = (
        (stock_daily['is_bullish'] == 0) & 
        (stock_daily['is_bullish'].shift(1) == 0) & 
        (stock_daily['is_bullish'].shift(2) == 0) &  # Three consecutive bearish candles
        (stock_daily['pct_chg'] <= -0.01) & 
        (stock_daily['pct_chg'].shift(1) <= -0.01) & 
        (stock_daily['pct_chg'].shift(2) <= -0.01) &  # Daily decline ≥1%
        (stock_daily['body'] >= 0.75) & 
        (stock_daily['body'].shift(1) >= 0.75) & 
        (stock_daily['body'].shift(2) >= 0.75) &  # Body ratio ≥75%
        (stock_daily['open'].shift(2) > stock_daily['close'].shift(3)) &  # Gap up two days prior
        (stock_daily['pct_chg'].shift(3) > 0)  # Positive return three days prior
    )

    # High price level condition
    condition_high = (stock_daily['is_high'] == 1) 

    all_stock_daily.loc[idx[:,stock],'three_crows_high'] = (
        condition_crows & condition_high
    ).astype(int)

# Filter three black crows with high level records
three_crows_high_records = all_stock_daily[all_stock_daily['three_crows_high'] == 1]

# Group by date and collect stock codes
daily_three_crows_high_stocks = three_crows_high_records.groupby(level='trade_date').apply(
    lambda x: x.index.get_level_values('ts_code').tolist()
)

In [135]:
# Three Black Crows with High Level Performance
result_three_crows_high = backtest_strategy(daily_three_crows_high_stocks, all_stock_price_daily, volume_price_state="Three Black Crows + High Level")
result_three_crows_high

,Price-Volume State,Holding Period,Average Return,Return Std Dev,Max Gain,Max Loss,Median Return,Win Rate
0,Three Black Crows + High Level,Hold 05 days,5.39%,0.32,150.95%,-72.23%,0.37%,51.89%
1,Three Black Crows + High Level,Hold 10 days,4.60%,0.42,138.04%,-64.14%,-3.14%,44.66%
2,Three Black Crows + High Level,Hold 20 days,15.51%,0.73,439.39%,-70.92%,-3.25%,45.83%
